In [ ]:
# 0. Colab 파일 업로드 (한 번만 필요)
from google.colab import files
print("필요 파일 업로드")
uploaded = files.upload()

필요 파일 업로드


Saving bus_route.xlsx to bus_route.xlsx
Saving bus_stops.xlsx to bus_stops.xlsx
Saving metro_inside.json to metro_inside.json
Saving metro_station.json to metro_station.json
Saving metro_line.json to metro_line.json
Saving metro_transfer_time .json to metro_transfer_time .json


In [ ]:
!pip install folium pandas networkx osmnx scipy geopy openpyxl cch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.0/338.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 130.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 1. 라이브러리 설치 및 import
import json
import os
import pandas as pd
import networkx as nx
import osmnx as ox
import numpy as np
from osmnx.distance import nearest_nodes
from scipy.spatial import cKDTree
from geopy.geocoders import Nominatim
from itertools import combinations, islice
from math import radians, sin, cos, sqrt, asin
from multiprocessing import Pool
import cch
from geopy.geocoders import Nominatim
from concurrent.futures import ProcessPoolExecutor
import heapq
from itertools import product
from collections import defaultdict
from datetime import datetime, time

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 2. 실제 도로망 전체 로드 (network_type='drive')
#    - 서울 전역을 대상으로 OSMnx의 drive 네트워크를 가져옵니다.
#    - 로그가 너무 많으면 아래 주석을 해제하고 설정을 바꿀 수 있습니다.
# ox.config(log_console=True, use_cache=True)

road_graph = ox.graph_from_place("Seoul, South Korea", network_type="drive")

# 2.1. ROAD_COORD 캐시에 각 노드의 좌표(위도/경도) 저장
ROAD_COORD = {}
for node, data in road_graph.nodes(data=True):
    ROAD_COORD[node] = (data['y'], data['x'])

print("도로망 로드 완료")
print("  - 도로망 노드 수:", road_graph.number_of_nodes())
print("  - 도로망 엣지 수:", road_graph.number_of_edges())

도로망 로드 완료
  - 도로망 노드 수: 68314
  - 도로망 엣지 수: 190011


나중에 사용자 위치도 매핑 시켜야함. walk 그래프에. 다만 아직 지도 위에 찍지는 않고, 콘솔에 좌표만 출력하므로 “지도에 매핑” 단계는 다음에 Folium Map 객체 생성 및 Marker 추가를 해주셔야 합니다.

In [ ]:
# Step1: 사용자 출발지, 도착지 입력 및 지오코딩

# 1.1 geolocator 초기화
geolocator = Nominatim(user_agent="route_planner")

# 1.2 지오코딩 함수 정의
def get_location(name):
    loc = geolocator.geocode(name, timeout=10)
    if not loc:
        loc = geolocator.geocode(f"{name}, Seoul", timeout=10)
    if not loc:
        raise ValueError(f"'{name}' 위치를 찾을 수 없습니다.")
    return loc.latitude, loc.longitude

# 1.3 사용자에게 출발지와 도착지 명칭 입력받기
start_name = input("출발지 (예: 고려대학교): ")
end_name   = input("도착지 (예: 한남힐스테이트): ")

# 1.4 입력값을 위경도로 변환
start_lat, start_lon = get_location(start_name)
end_lat,   end_lon   = get_location(end_name)

# 1.5 결과 확인용 출력
print(f"출발지 '{start_name}' -> 위도: {start_lat:.6f}, 경도: {start_lon:.6f}")
print(f"도착지 '{end_name}' -> 위도: {end_lat:.6f},   경도: {end_lon:.6f}")

출발지 (예: 고려대학교): 종로3가역
도착지 (예: 한남힐스테이트): 강남역
출발지 '종로3가역' -> 위도: 37.570440, 경도: 126.992324
도착지 '강남역' -> 위도: 37.499935,   경도: 127.026968


ㄱHaversine 직선 거리(미터): 10065.997667197036


In [ ]:
# 예시 좌표 (출발/도착 좌표)
start_lat, start_lon = 37.570440, 126.992324
end_lat,   end_lon   = 37.497541, 127.026883

u = ox.distance.nearest_nodes(road_graph, start_lon, start_lat)
v = ox.distance.nearest_nodes(road_graph, end_lon,   end_lat)
print("출발 노드 ID:", u, "도착 노드 ID:", v)


출발 노드 ID: 12234274109 도착 노드 ID: 10220955579


In [ ]:
# 노드가 제대로 선택되었다면 실제 최단 경로 길이를 구해 본다
try:
    length_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    print("도로망 최단경로 길이(미터):", length_m)
except nx.NetworkXNoPath:
    print("두 노드가 같은 연결 성분에 속해 있지 않습니다. (No path)")


도로망 최단경로 길이(미터): 9845.427578064353


In [ ]:
# 2. 유틸리티 함수 정의
def haversine(lat1, lon1, lat2, lon2):
    """두 지점 간 대원거리 계산 (미터)"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371000 * c

def road_distance(lat1, lon1, lat2, lon2):
    """도로망 최단 경로 길이(미터) 또는 haversine fallback"""
    try:
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        return nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        return haversine(lat1, lon1, lat2, lon2)

In [ ]:
dist = road_distance(start_lat, start_lon, end_lat, end_lon)
print("도로망 기준 거리(미터):", dist)

도로망 기준 거리(미터): 9845.427578064353


In [ ]:
# 지하철 관련 전역 변수 (Step3에서 채울 예정)
subway_info = {}    # { station_name: {'lat': float, 'lng': float, 'lines': set([...])}, ... }
subway_graph = {}   # { (station_name, line): [ (neighbor_station, neighbor_line, weight), ... ], ... }

# 버스 관련 전역 변수 (Step4에서 채울 예정)
bus_info = {}       # { stop_id: {'name': str, 'lat': float, 'lng': float}, ... }
bus_graph = {}      # { stop_id: [ (neighbor_stop_id, weight), ... ], ... }

# 글로벌 속도 및 패널티 설정
AVG_SPEED_KMH_GLOBAL = 18.0   # 대중교통 평균 속도 약 18 km/h (필요시 조정)
WALK_SPEED = 5.0              # 보행 속도 5 km/h
WALK_PENALTY = 3.0            # 환승 시 부과되는 보행+대기 패널티 3분


경강선, 인천1호선,인천2호선, 서해선,김포골드라인,용인에버라인,의정부경전철,GTXA,경춘선 제거.

In [ ]:
# 1~9호선 및 기타 전철·경전철 노선별로 직접 지정
speed_by_line = {
    '1호선': 28.6, '2호선': 36.5, '3호선': 34.9, '4호선': 30.1,
    '5호선': 32.8, '6호선': 30.0, '7호선': 32.3, '8호선': 32.2,
    '9호선': 26.2,
    '경의중앙선': 41.5, '수인분당선': 41.1,
    '신분당선': 54.9, '신림선': 26.5, '우이신설선': 28.7,
}

In [ ]:
# 전역으로 이미 로드되어 있어야 할 것들:
#   road_graph  ──> OSMnx로 만든 도로망 그래프 (networkx.Graph or MultiDiGraph)
#   haversine   ──> (lat1, lon1, lat2, lon2) 간 거리(미터) 계산 함수
#   WALK_SPEED, WALK_PENALTY ──> 보행 속도(km/h), 환승 페널티(분) 등

def build_metro_graph(
    station_json: str,
    line_json: str,
    speed_by_line: dict
):
    """
    station_json: 지하철 역 정보 JSON 파일 경로
    line_json:    지하철 노선 연결 정보 JSON 파일 경로
    speed_by_line: { '1호선': 30.0, '2호선': 32.0, ... }처럼 호선별 평균 속도( km/h ) 딕셔너리

    - station_json의 각 원소 s는 최소한 {'station_cd','name','lat','lng'} 키를 가져야 합니다.
    - line_json의 각 원소 info는 {'line':호선명, 'node':[ {'station':[ {...}, {...} ]}, ... ]} 구조이어야 합니다.
    """
    # 1) JSON 데이터 로드
    stations = json.load(open(station_json, 'r', encoding='utf-8'))['DATA']
    lines    = json.load(open(line_json,    'r', encoding='utf-8'))['DATA']

    # JSON 로드 후, speed_by_line에 정의된 호선만 남긴다
    lines = [info for info in lines if info['line'] in speed_by_line]

    # 2) station_cd → {name, lat, lng} 사전 생성
    station_map = {
        s['station_cd']: {
            'name': s.get('name', s.get('station_nm')),
            'lat':  float(s['lat']),
            'lng':  float(s['lng'])
        }
        for s in stations
    }

    # 3) 남은 lines에 등장하는 station_cd만 사용하도록 필터링
    used_ids = {
        seg['station'][0]['station_cd']
        for info in lines
        for seg in info.get('node', [])
    } | {
        seg['station'][1]['station_cd']
        for info in lines
        for seg in info.get('node', [])
    }
    station_ids = list(used_ids)

    # 4) KDTree용 좌표 배열 생성 (사용자 좌표 → nearest 역 인덱스 용)
    coords_metro = np.array([
        [station_map[_id]['lat'], station_map[_id]['lng']]
        for _id in station_ids
    ])
    station_tree = cKDTree(coords_metro)

    # 5) 도로망 그래프 상의 노드 매핑: station_cd → nearest road_graph 노드 ID
    lats = [station_map[_id]['lat'] for _id in station_ids]
    lons = [station_map[_id]['lng'] for _id in station_ids]
    road_nodes = nearest_nodes(road_graph, lons, lats)
    station_to_road = dict(zip(station_ids, road_nodes))

    # 6) Graph 초기화
    Gm = nx.Graph()

    # 7) 호선별 인접역 엣지 추가
    for info in lines:
        line_name = info['line']
        speed_kmh = speed_by_line.get(line_name, 30.0)
        for seg in info.get('node', []):
            a_cd = seg['station'][0]['station_cd']
            b_cd = seg['station'][1]['station_cd']
            na = station_map[a_cd]
            nb = station_map[b_cd]
            u_node = f"{line_name}_{na['name']}"
            v_node = f"{line_name}_{nb['name']}"

            # 노드 추가
            if not Gm.has_node(u_node):
                Gm.add_node(u_node, station=na['name'], line=line_name, lat=na['lat'], lng=na['lng'])
            if not Gm.has_node(v_node):
                Gm.add_node(v_node, station=nb['name'], line=line_name, lat=nb['lat'], lng=nb['lng'])

            # 도로망 기반 거리 계산
            u_rg = station_to_road[a_cd]
            v_rg = station_to_road[b_cd]
            try:
                dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
            except Exception:
                dist_m = haversine(na['lat'], na['lng'], nb['lat'], nb['lng'])

            # 시간 weight 계산 (분)
            t_min = dist_m / 1000 / speed_kmh * 60
            Gm.add_edge(u_node, v_node, weight=t_min, mode='metro_edge')

    # 8) 환승 엣지 추가 (같은 station_cd에 따른 역 연결)
    st2nodes = defaultdict(list)
    for node, data in Gm.nodes(data=True):
        st2nodes[data['station']].append(node)

    for nodes_list in st2nodes.values():
        if len(nodes_list) < 2:
            continue
        for i in range(len(nodes_list)):
            for j in range(i+1, len(nodes_list)):
                u, v = nodes_list[i], nodes_list[j]
                lat_u, lng_u = Gm.nodes[u]['lat'], Gm.nodes[u]['lng']
                lat_v, lng_v = Gm.nodes[v]['lat'], Gm.nodes[v]['lng']
                walk_m = haversine(lat_u, lng_u, lat_v, lng_v)
                wt = walk_m / 1000 / WALK_SPEED * 60 + WALK_PENALTY
                Gm.add_edge(u, v, weight=wt, mode='transfer')
                Gm.add_edge(v, u, weight=wt, mode='transfer')

    return Gm, station_ids, station_tree


In [ ]:
# 올바른 언패킹 예시
metro_graph, station_ids, station_tree = build_metro_graph(
    'metro_station.json',
    'metro_line.json',
    speed_by_line
)

print(f"지하철 그래프 노드 개수: {metro_graph.number_of_nodes()}")
print(f"지하철 그래프 엣지 개수: {metro_graph.number_of_edges()}")


지하철 그래프 노드 개수: 612
지하철 그래프 엣지 개수: 712


In [ ]:
# 1-1) 노드 정보 추출 & DataFrame 생성
nodes_data = [
    {
        'node_key': node,
        'station': data['station'],
        'line':    data['line'],
        'lat':     data['lat'],
        'lng':     data['lng']
    }
    for node, data in metro_graph.nodes(data=True)
]
metro_nodes_df = pd.DataFrame(nodes_data)
print(f"총 메트로 노드 수: {len(metro_nodes_df)}")
display(metro_nodes_df.head(10))

# 1-2) 엣지 정보 추출 & DataFrame 생성
edges_data = [
    {
        'from':       u,
        'to':         v,
        'mode':       data.get('mode'),
        'weight_min': data.get('weight')
    }
    for u, v, data in metro_graph.edges(data=True)
]
metro_edges_df = pd.DataFrame(edges_data)
print(f"총 메트로 엣지 수: {len(metro_edges_df)}")
display(metro_edges_df.head(10))

# 1-3) 특정 역 연결 확인 (예: '신당'역)
station_of_interest = '신당'
mask = metro_nodes_df['station'] == station_of_interest
nodes_of_interest = metro_nodes_df.loc[mask, 'node_key'].tolist()

print(f"\n'{station_of_interest}'역에 해당하는 그래프 노드들:", nodes_of_interest)
for nk in nodes_of_interest:
    nbrs = metro_graph[nk]
    print(f"  • {nk}와 연결된 노드:")
    for nbr, attr in nbrs.items():
        print(f"      - {nbr} (mode={attr['mode']}, {attr['weight']:.2f}분)")


총 메트로 노드 수: 612


,node_key,station,line,lat,lng
0,1호선_연천,연천,1호선,38.100730,127.073720
1,1호선_전곡,전곡,1호선,38.024580,127.071800
2,1호선_청산,청산,1호선,37.981720,127.069120
3,1호선_소요산,소요산,1호선,37.948100,127.061034
4,1호선_동두천,동두천,1호선,37.927878,127.054790
5,1호선_보산,보산,1호선,37.913702,127.057277
6,1호선_동두천중앙,동두천중앙,1호선,37.901885,127.056482
7,1호선_지행,지행,1호선,37.892334,127.055716
8,1호선_덕정,덕정,1호선,37.843188,127.061277
9,1호선_덕계,덕계,1호선,37.818486,127.056486


총 메트로 엣지 수: 712


,from,to,mode,weight_min
0,1호선_연천,1호선_전곡,metro_edge,0.0
1,1호선_전곡,1호선_청산,metro_edge,0.0
2,1호선_청산,1호선_소요산,metro_edge,0.0
3,1호선_소요산,1호선_동두천,metro_edge,0.0
4,1호선_동두천,1호선_보산,metro_edge,0.0
5,1호선_보산,1호선_동두천중앙,metro_edge,0.0
6,1호선_동두천중앙,1호선_지행,metro_edge,0.0
7,1호선_지행,1호선_덕정,metro_edge,0.0
8,1호선_덕정,1호선_덕계,metro_edge,0.0
9,1호선_덕계,1호선_양주,metro_edge,0.0



'신당'역에 해당하는 그래프 노드들: ['2호선_신당', '6호선_신당']
  • 2호선_신당와 연결된 노드:
      - 2호선_상왕십리 (mode=metro_edge, 1.47분)
      - 2호선_동대문역사문화공원 (mode=metro_edge, 1.87분)
      - 6호선_신당 (mode=transfer, 6.73분)
  • 6호선_신당와 연결된 노드:
      - 6호선_청구 (mode=metro_edge, 1.34분)
      - 6호선_동묘앞 (mode=metro_edge, 1.28분)
      - 2호선_신당 (mode=transfer, 6.73분)


In [ ]:
def compute_bus_edge(task):
    """
    task: (u_idx, v_idx, lines_set)
    STOP_COORD, STOP_TO_ROAD, road_graph, AVG_SPEED_KMH_GLOBAL 전역 사용.

    # 버스 구간도 OSM 도로망 기반 최단경로로 계산하도록 변경
    """
    u_idx, v_idx, lines = task

    # 1) 허버사인 대신, ‘정류장 idx → 가장 가까운 도로망 노드’ 매핑 사용
    u_rg = STOP_TO_ROAD[u_idx]
    v_rg = STOP_TO_ROAD[v_idx]

    # 2) OSM 도로망에서 최단 경로 길이(미터) 구하기 (weight='length')
    try:
        dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
    except (nx.NetworkXNoPath, Exception):
        # 도로망 경로가 없으면 허버사인으로 fallback
        y1, x1 = STOP_COORD[u_idx]
        y2, x2 = STOP_COORD[v_idx]
        dist_m = haversine(y1, x1, y2, x2)

    # 3) 거리 → 버스 평균 속도(AVG_SPEED_KMH_GLOBAL)로 분 단위 환산
    t_min = dist_m / 1000 / AVG_SPEED_KMH_GLOBAL * 60
    return u_idx, v_idx, t_min, lines


In [ ]:
 def build_bus_graph_optimized(
    stops_excel: str,
    routes_excel: str,
    avg_speed_kmh: float = 18.0
):
    """
    1) stops_excel: 'bus_stops_seoul.xlsx' 경로 (컬럼: ['정류소명','X좌표','Y좌표'])
    2) routes_excel: 'bus_route.xlsx' 경로 (컬럼: ['노선명','정류소명','순번','X좌표','Y좌표'])
    3) avg_speed_kmh: 버스 평균 속도(km/h), 기본 18.0

    - 버스 정류장 간 좌표 매핑 → 잘못 매핑된(거리 차이 큰) 로우들은 자동으로 드롭
    - STOP_TO_ROAD: 각 정류장 인덱스를 OSM 도로망 노드 ID로 매핑하고,
      동일 도로망 노드에 매핑된 정류장이 몇 개 있는지(overlap)를 출력
    - 노드명 = f"{노선명}_{정류소명}"
    - 버스 이동 엣지 생성 (OSM 도로망 최단경로 이용)
    - 보행 환승 엣지 생성 (1,000m 이내, 20분 이내 걸어갈 수 있으면)
    - 반환값: (bus_graph, edge_lines)
      • bus_graph: networkx.Graph (nodes: 노선명_정류소명, edges: mode, weight)
      • edge_lines: { (u_node, v_node): set(노선명) }
    """

    global STOP_COORD, STOP_TO_ROAD, AVG_SPEED_KMH_GLOBAL
    AVG_SPEED_KMH_GLOBAL = avg_speed_kmh

    # ─────────────────────────────────────────────────────────────────────────
    # 1) 정류장 엑셀 불러오기 & KDTree 생성
    stops_df = pd.read_excel(stops_excel, dtype={'X좌표': float, 'Y좌표': float})
    required_cols = {'정류소명','X좌표','Y좌표'}
    if not required_cols.issubset(stops_df.columns):
        raise ValueError(f"stops_excel에 반드시 {required_cols} 컬럼이 있어야 합니다.")

    # 인덱스가 정류장 고유 ID 역할 (0, 1, 2, …)
    stops_df = stops_df.reset_index(drop=True)

    # (1-1) KDTree 생성: stops_df의 (위도, 경도) 배열
    coords_bus = stops_df[['Y좌표','X좌표']].to_numpy()  # shape=(N,2)
    stop_tree  = cKDTree(coords_bus)

    # ─────────────────────────────────────────────────────────────────────────
    # 2) 노선 정보 엑셀 불러오기 & 좌표 기반 매핑 → 잘못된 매핑(drop)
    routes_df = pd.read_excel(routes_excel, dtype={'X좌표': float, 'Y좌표': float})
    required_cols2 = {'버스번호','정류소명','순번','X좌표','Y좌표'}
    if not required_cols2.issubset(routes_df.columns):
        raise ValueError(f"routes_excel에 반드시 {required_cols2} 컬럼이 있어야 합니다.")

    # (2-1) routes_df 각 행의 (위도,경도)를 KDTree 로 쿼리 → nearest stops_df 인덱스 찾기
    route_coords = routes_df[['Y좌표','X좌표']].to_numpy()
    dists, nearest_idx = stop_tree.query(route_coords, k=1)
    routes_df['matched_stop_idx'] = nearest_idx.astype(int)
    routes_df['matched_dist_m'] = [
        haversine(lat1, lon1,
                  stops_df.loc[idx,'Y좌표'], stops_df.loc[idx,'X좌표'])
        for (lat1, lon1), idx in zip(route_coords, nearest_idx)
    ]

    # (2-2) 특정 거리 임계치 이상(여기서는 50m) 차이가 나는 경우 “잘못된 좌표”로 보고 드롭
    DROP_THRESHOLD_M = 50.0
    mask_good = routes_df['matched_dist_m'] <= DROP_THRESHOLD_M
        # ↓ 여기에 삽입 ↓
    # KDTree 매핑 거리가 50m 초과한 로우(드롭될 행) 미리 살펴보기
    unassigned = routes_df[~mask_good]
    print("=== KDTree 매핑 거리가 50m 초과하여 드롭될 예시 10개 ===")
    # Jupyter가 아니면 print()만 사용해도 무방합니다
    display(unassigned[['버스번호','정류소명','matched_dist_m']].head(10))
    print(f"총 드롭된 로우 수: {len(unassigned)}")
    # ↑ 여기까지 삽입 ↑
    num_dropped = (~mask_good).sum()
    if num_dropped > 0:
        print(f">>> **{num_dropped}개의 노선-정류장 로우는 KDTree 매핑 거리가 {DROP_THRESHOLD_M}m 초과하여 드롭됩니다.**")
    routes_df = routes_df[mask_good].copy()

    # ─────────────────────────────────────────────────────────────────────────
    # 3) 버스 그래프 초기화 & 노드 생성 준비
    bus_graph = nx.Graph()
    # edge_lines: “(u_node, v_node) → {노선명 set}”
    edge_lines = defaultdict(set)

    # STOP_COORD: 정류장 idx → (lat, lon)
    STOP_COORD = {
        idx: (row['Y좌표'], row['X좌표'])
        for idx, row in stops_df.iterrows()
    }

    # ─────────────────────────────────────────────────────────────────────────
    # 4) STOP_TO_ROAD (정류장 idx → OSM 도로망 노드 ID) 매핑
    #    (lon, lat) 순서로 nearest_nodes 호출
    lats = stops_df['Y좌표'].values
    lons = stops_df['X좌표'].values
    road_nodes = ox.distance.nearest_nodes(road_graph, lons, lats)
    STOP_TO_ROAD = dict(enumerate(road_nodes))

    # 4-1) 동일한 OSM 노드에 매핑된 정류장(들) 찾기 → overlap 체크
    inv = defaultdict(list)
    for stop_idx, rnode in STOP_TO_ROAD.items():
        inv[rnode].append(stop_idx)

    overlap_count = 0
    overlap_groups = []
    for rnode, idx_list in inv.items():
        if len(idx_list) > 1:
            overlap_groups.append(idx_list)
            overlap_count += len(idx_list)
    if overlap_count > 0:
        print(f">>> **OSM 노드 상에서 겹치는(같은 노드에 매핑된) 정류장 인덱스 총 {overlap_count}개**")
        print("   - 예시 겹침 그룹 (stop_idx 리스트):")
        for grp in overlap_groups:
            print(f"     → {grp}")

    # ─────────────────────────────────────────────────────────────────────────
    # 5) “노선명_정류소명” 노드 & 버스 엣지 생성
    #    → 각 노선별로 ‘순번’ 순으로 정차 정류장 인덱스를 정렬한 뒤 인접 쌍(u_idx, v_idx)에 대해 엣지 추가
    for line_name, grp in routes_df.groupby('버스번호'):
        grp_sorted = grp.sort_values('순번')
        prev_idx = None
        for _, row in grp_sorted.iterrows():
            cur_idx = int(row['matched_stop_idx'])
            stop_name = stops_df.loc[cur_idx, '정류소명']

            # 노드명: "노선명_정류소명"
            this_node = f"{line_name}_{stop_name}"

            # 실제 노드가 없으면 생성 (노드 속성: coord, stop_idx, 노선명)
            if not bus_graph.has_node(this_node):
                lat0, lon0 = STOP_COORD[cur_idx]
                bus_graph.add_node(
                    this_node,
                    coord     = (lat0, lon0),
                    stop_idx  = cur_idx,
                    route     = line_name
                )

            if prev_idx is not None and prev_idx != cur_idx:
                prev_stop_name = stops_df.loc[prev_idx, '정류소명']
                prev_node = f"{line_name}_{prev_stop_name}"

                # 5-1) 버스 구간 거리 계산 (OSM 도로망 경로 길이)
                u_rg = STOP_TO_ROAD[prev_idx]
                v_rg = STOP_TO_ROAD[cur_idx]
                try:
                    dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
                except (nx.NetworkXNoPath, Exception):
                    # 도로망 연결 없으면 직선거리 fallback
                    lat_u, lon_u = STOP_COORD[prev_idx]
                    lat_v, lon_v = STOP_COORD[cur_idx]
                    dist_m = haversine(lat_u, lon_u, lat_v, lon_v)

                # 5-2) 소요 시간 계산(분) = 거리(m) / 1000 / 속도(km/h) * 60
                t_min = dist_m / 1000 / AVG_SPEED_KMH_GLOBAL * 60

                # 5-3) 양방향 버스 엣지 추가
                bus_graph.add_edge(
                    prev_node, this_node,
                    weight = t_min,
                    mode   = 'bus_edge'
                )
                bus_graph.add_edge(
                    this_node, prev_node,
                    weight = t_min,
                    mode   = 'bus_edge'
                )

                # 5-4) edge_lines에 해당 엣지를 어떤 노선이 쓰는지 기록
                edge_lines[(prev_node, this_node)].add(line_name)
                edge_lines[(this_node, prev_node)].add(line_name)

            prev_idx = cur_idx

    # ─────────────────────────────────────────────────────────────────────────
    # 6) 보행 환승 엣지 추가 (1,000m 반경 / 20분 이내)
    TRANSFER_RADIUS_M = 1000.0
    # 반경(deg) 단순 변환 (위도 1° ≈ 111 km) → 대략적인 검색용
    radius_deg = TRANSFER_RADIUS_M / 111000.0

    # stop_tree는 1번에서 생성한 KDTree
    # coords_bus: (위도,경도) 배열
    for u_idx in range(len(stops_df)):
        # 이 정류장(u_idx) 반경 radius_deg 내 이웃 정류장 인덱스 리스트
        nbrs = stop_tree.query_ball_point(coords_bus[u_idx], r=radius_deg)
        for v_idx in nbrs:
            if v_idx <= u_idx:
                continue

            # 실제 직선 거리(미터)
            lat_u, lon_u = STOP_COORD[u_idx]
            lat_v, lon_v = STOP_COORD[v_idx]
            dist_uv = haversine(lat_u, lon_u, lat_v, lon_v)

            # “걸어서 걸리는 시간(분)”
            walk_time = dist_uv / 1000 / WALK_SPEED * 60

            if walk_time <= 20.0:  # 20분 이내(≒1km)라면 환승 가능
                # u_idx, v_idx에 해당하는 모든 “노선별 노드”를 서로 연결
                # (stop_idx → 어떤 route들이 쓰고 있는지 알아야 함)
                # 따라서 먼저 “이 정류장 u_idx를 쓰는 노드들”을 찾자
                u_nodes = [node for node, data in bus_graph.nodes(data=True)
                           if data.get('stop_idx') == u_idx]
                v_nodes = [node for node, data in bus_graph.nodes(data=True)
                           if data.get('stop_idx') == v_idx]

                for un in u_nodes:
                    for vn in v_nodes:
                        # 이미 버스 엣지가 있으면 skip (버스 엣지가 있으면 mode='bus_edge'이므로)
                        if bus_graph.has_edge(un, vn):
                            continue
                        # u→v 환승 엣지
                        bus_graph.add_edge(
                            un, vn,
                            weight = walk_time + WALK_PENALTY,
                            mode   = 'transfer'
                        )
                        # v→u 환승 엣지
                        bus_graph.add_edge(
                            vn, un,
                            weight = walk_time + WALK_PENALTY,
                            mode   = 'transfer'
                        )

    return bus_graph, edge_lines


In [ ]:
# 4.3. build_bus_graph_optimized 함수 실행 및 결과 확인
bus_graph, edge_lines = build_bus_graph_optimized('bus_stops.xlsx', 'bus_route.xlsx', avg_speed_kmh=18.0)

print(f"최종 버스 그래프 노드(정류장) 개수: {bus_graph.number_of_nodes()}")
print(f"최종 버스 그래프 엣지(유니크) 개수: {bus_graph.number_of_edges()}")

=== KDTree 매핑 거리가 50m 초과하여 드롭될 예시 10개 ===


,버스번호,정류소명,matched_dist_m
27,용산01,순천향병원,77.398036
105,5713,비산체육공원,4073.631936
106,5626,비산체육공원,4073.631936
107,5625,비산체육공원,4073.631936
109,702A,선진운수원당차고지.원신4통입구,5703.687423
110,702B,선진운수원당차고지.원신4통입구,5703.687423
114,서초18,선바위역1번출구,1245.482371
115,1152,청구아파트,6430.276762
146,703,자이안트부대,25225.453933
148,7727,설문동,14223.245066


총 드롭된 로우 수: 3230
>>> **3230개의 노선-정류장 로우는 KDTree 매핑 거리가 50.0m 초과하여 드롭됩니다.**
>>> **OSM 노드 상에서 겹치는(같은 노드에 매핑된) 정류장 인덱스 총 6454개**
   - 예시 겹침 그룹 (stop_idx 리스트):
     → [1, 117, 232, 366, 369, 371, 376]
     → [8, 45]
     → [12, 262]
     → [20, 268]
     → [26, 28]
     → [27, 37]
     → [29, 34]
     → [30, 33]
     → [31, 32]
     → [35, 199, 271, 381]
     → [38, 51]
     → [39, 50, 289, 290]
     → [40, 264]
     → [43, 143, 380, 382]
     → [48, 281]
     → [54, 99, 378]
     → [55, 68, 363]
     → [59, 64]
     → [65, 148]
     → [69, 75]
     → [70, 132]
     → [71, 72]
     → [73, 362]
     → [78, 83]
     → [85, 167]
     → [87, 211]
     → [88, 5110]
     → [89, 543]
     → [90, 5092]
     → [91, 98, 293]
     → [100, 101, 142, 276, 372]
     → [102, 373]
     → [111, 116]
     → [112, 144, 302]
     → [113, 114, 375]
     → [115, 227, 365, 370]
     → [120, 249]
     → [121, 243, 364]
     → [145, 296]
     → [147, 253]
     → [151, 2030]
     → [155, 156, 291, 292]
     → [160,

In [ ]:
# 예시: "bus_2840" 와 "bus_9421" 의 정류장 인덱스(idx) 가 각각 무엇인지 알아낸 뒤 그 값을 넣어서 테스트

# 1) 정류장 idx → road node ID 출력
print("=== STOP_TO_ROAD 샘플 매핑 확인 ===")
sample_indices = [  # 여기에는 실제 버스 노드 인덱스를 넣어주세요. 예제에서는 2840, 9421 가 아니라
                   # stops_df 에서 할당된 idx(0~n-1) 값이어야 합니다.
                   # 코드 전체에서 가장 먼저 'bus_2840' 처럼 쓰인 건 "bus_graph 노드 키"이지,
                   # STOP_TO_ROAD에서는 정수 idx 그대로를 사용했으니,
                   # stops_df 내부에서 '정류장ID'가 몇 번째 인덱스인지 찾아서 써야 합니다.
                   2840, 9421
                 ]
for idx in sample_indices:
    if idx in STOP_TO_ROAD:
        print(f"정류장 idx {idx} → road_node ID: {STOP_TO_ROAD[idx]}")
    else:
        print(f"정류장 idx {idx} 은 STOP_TO_ROAD에 없습니다.")

# 2) 만약 동일 ID가 찍히면, 두 정류장은 사실상 road_graph 상 같은 노드를 가리키고 있는 것
if len(sample_indices) == 2:
    u_rg = STOP_TO_ROAD.get(sample_indices[0])
    v_rg = STOP_TO_ROAD.get(sample_indices[1])
    print("두 정류장 road-node 동일 여부:", u_rg == v_rg)


=== STOP_TO_ROAD 샘플 매핑 확인 ===
정류장 idx 2840 → road_node ID: 7579130762
정류장 idx 9421 → road_node ID: 10220955579
두 정류장 road-node 동일 여부: False


In [ ]:
import pandas as pd

# 2-1) 노드 정보 추출 & DataFrame 생성
nodes_data = [
    {
        'node_key': node,
        'stop_idx': data['stop_idx'],
        'route':    data['route'],
        'lat':      data['coord'][0],
        'lng':      data['coord'][1]
    }
    for node, data in bus_graph.nodes(data=True)
]
bus_nodes_df = pd.DataFrame(nodes_data)
print(f"총 버스 노드 수: {len(bus_nodes_df)}")
display(bus_nodes_df.head(10))

# 2-2) 엣지 정보 추출 & DataFrame 생성
edges_data = [
    {
        'from':       u,
        'to':         v,
        'mode':       data.get('mode'),
        'weight_min': data.get('weight')
    }
    for u, v, data in bus_graph.edges(data=True)
]
bus_edges_df = pd.DataFrame(edges_data)
print(f"총 버스 엣지 수: {len(bus_edges_df)}")
display(bus_edges_df.head(10))

# 2-3) 특정 정류장 연결 확인 (예: 6번째 행의 stop_idx)
stop_idx_of_interest = bus_nodes_df.loc[5, 'stop_idx']

# ① stop_idx로 해당하는 모든 node_key를 뽑아낸 뒤
nodes_of_interest = bus_nodes_df.loc[
    bus_nodes_df['stop_idx'] == stop_idx_of_interest, 'node_key'
].tolist()

print(f"정류장 idx={stop_idx_of_interest}에 해당하는 노드들:", nodes_of_interest)

# ② 실제 존재하는 node_key로 bus_graph 조회
for node_key in nodes_of_interest:
    print(f"\n— {node_key}의 연결 목록 —")
    for nbr, attr in bus_graph[node_key].items():
        print(f"  • {nbr} (mode={attr['mode']}, {attr['weight']:.2f}분)")



총 버스 노드 수: 26640


,node_key,stop_idx,route,lat,lng
0,0017_신용산지하차도,762,0017,37.532590,126.963882
1,0017_용산역,764,0017,37.528867,126.965822
2,0017_용산푸르지오써밋,765,0017,37.527576,126.964760
3,0017_한강대교북단,633,0017,37.524969,126.963571
4,0017_서부이촌동입구,802,0017,37.523864,126.959495
5,0017_이촌2동대림아파트.새남터성지,800,0017,37.526175,126.956323
6,0017_이촌2동주민센터,798,0017,37.527736,126.955064
7,0017_성촌공원.포르쉐센터용산,796,0017,37.530171,126.954607
8,0017_이촌119안전센터.성촌공원,795,0017,37.530080,126.952947
9,0017_원효2동산호아파트후문,804,0017,37.532260,126.950013


총 버스 엣지 수: 2193796


,from,to,mode,weight_min
0,0017_신용산지하차도,0017_용산역,bus_edge,2.012015
1,0017_신용산지하차도,0017_용산전자상가입구,bus_edge,0.713411
2,0017_신용산지하차도,0017_신용산역3번출구,bus_edge,2.995241
3,0017_신용산지하차도,0017_한강대교북단.LG유플러스,transfer,14.423753
4,0017_신용산지하차도,040_한강대교북단.LG유플러스,transfer,14.423753
5,0017_신용산지하차도,500_한강대교북단.LG유플러스,transfer,14.423753
6,0017_신용산지하차도,501_한강대교북단.LG유플러스,transfer,14.423753
7,0017_신용산지하차도,504_한강대교북단.LG유플러스,transfer,14.423753
8,0017_신용산지하차도,506_한강대교북단.LG유플러스,transfer,14.423753
9,0017_신용산지하차도,507_한강대교북단.LG유플러스,transfer,14.423753


정류장 idx=800에 해당하는 노드들: ['0017_이촌2동대림아파트.새남터성지', '2016_이촌2동대림아파트.새남터성지']

— 0017_이촌2동대림아파트.새남터성지의 연결 목록 —
  • 0017_서부이촌동입구 (mode=bus_edge, 1.03분)
  • 0017_이촌2동주민센터 (mode=bus_edge, 0.75분)
  • 0017_이촌2동강변아파트 (mode=bus_edge, 1.36분)
  • 0017_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 040_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 500_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 501_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 504_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 506_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 507_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 605_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 100_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 150_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 151_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 152_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 742_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 750A_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 750B_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 752_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • N15_한강대교북단.LG유플러스 (mode=trans

In [ ]:
# --- 5.0. 사전 준비: 필요한 전역 변수 및 함수가 이미 정의되었다고 가정 ---
# - metro_graph: Step3에서 생성된 지하철 그래프
# - bus_graph:   Step4에서 생성된 버스 그래프
# - STOP_COORD:  { idx: (lat, lon) } (버스 정류장 인덱스 → 위/경도)
# - WALK_SPEED, WALK_PENALTY: 보행 속도(km/h), 환승 패널티(분)
# - haversine(lat1, lon1, lat2, lon2): 두 지점 간 거리(미터) 계산 함수

# 5.1. 통합 그래프 초기화
integrated_graph = nx.Graph()


In [ ]:
# ─── 1. 지하철 노드·엣지 복사 ─────────────────────────────────────────
for node, data in metro_graph.nodes(data=True):
    integrated_graph.add_node(
        node,
        mode='metro',
        station=data.get('station'),
        line=data.get('line'),
        lat=data.get('lat'),
        lng=data.get('lng')
    )

for u, v, data in metro_graph.edges(data=True):
    w = data.get('weight')
    if data.get('mode') == 'transfer':
        integrated_graph.add_edge(u, v, weight=w, mode='transfer')
    else:
        integrated_graph.add_edge(u, v, weight=w, mode='metro_edge')


In [ ]:
 # ─── 2. 버스 노드·엣지 복사 ─────────────────────────────────────────
 # bus_graph 노드(key) 그대로 사용, 반드시 'route'까지 복사해야 합니다
for node, data in bus_graph.nodes(data=True):
    integrated_graph.add_node(
        node,
        node_key = node,
        mode     = 'bus',
        stop_idx = data['stop_idx'],
        route    = data['route'],
        lat      = data['coord'][0],
        lng      = data['coord'][1]
    )
for (u, v), lines in edge_lines.items():
    integrated_graph.add_edge(
        u, v,
        weight = bus_graph[u][v]['weight'],
        mode   = bus_graph[u][v].get('mode','bus_edge'),
        lines  = lines
    )

In [ ]:
# ─── 3. 버스↔지하철 환승 엣지 추가 (1 000m 반경) ────────────────────────

# 3-1) 버스 정류장 좌표 배열 + KDTree 준비
# STOP_COORD: { stop_idx: (lat, lon) } 형태
bus_indices   = list(STOP_COORD.keys())
bus_coords    = np.array([STOP_COORD[s] for s in bus_indices])  # shape=(N_bus, 2) → [(lat,lon), ...]
bus_tree      = cKDTree(bus_coords)

# (위도 1° ≈ 111 km) 정도이므로, 1 000 m 반경을 위/경도 단위로 _대략_ 변환
TRANSFER_RADIUS_M = 1000.0
radius_deg = TRANSFER_RADIUS_M / 111000.0  # ≈ 0.009009°

# 3-2) 각 메트로 노드마다 주변 버스 정류장 후보 찾고, 실제 거리 재확인
for m_node, m_data in metro_graph.nodes(data=True):
    metro_lat, metro_lon = m_data['lat'], m_data['lng']
    # KDTree 에서는 (lat, lon) 순서로 쿼리
    # 반경(radius_deg) 내 후보 버스 정류장 인덱스 배열(idx_in_bus_coords_list)을 돌려준다
    idxs_in_bus_array = bus_tree.query_ball_point([metro_lat, metro_lon], r=radius_deg)

    # 실제 환승 가능한 버스 정류장만 골라서 엣지 추가
    for arr_i in idxs_in_bus_array:
        stop_idx = bus_indices[arr_i]           # STOP_COORD의 실제 stop_idx
        bus_lat, bus_lon = STOP_COORD[stop_idx]

        # 실제 거리를 haversine으로 계산
        dist_m = haversine(bus_lat, bus_lon, metro_lat, metro_lon)
        if dist_m <= TRANSFER_RADIUS_M:
            walk_time     = (dist_m / 1000.0) / WALK_SPEED * 60.0
            transfer_time = walk_time + WALK_PENALTY

            bus_node_key  = f"bus_{stop_idx}"
            integrated_graph.add_edge(
                bus_node_key, m_node,
                weight = transfer_time,
                mode   = 'transfer'
            )


In [ ]:
# ─── 4. 결과 확인 ─────────────────────────────────────────────────
node_count     = integrated_graph.number_of_nodes()
edge_count     = integrated_graph.number_of_edges()
transfer_edges = [(u, v) for u, v, d in integrated_graph.edges(data=True) if d['mode'] == 'transfer']

print("통합 그래프 노드 개수:", node_count)
print("통합 그래프 엣지 개수:", edge_count)
print("버스↔지하철 환승 엣지 개수:", len(transfer_edges))


통합 그래프 노드 개수: 36707
통합 그래프 엣지 개수: 55030
버스↔지하철 환승 엣지 개수: 24578


In [ ]:
# 통합 그래프에서 버스/지하철 환승 엣지만 샘플 확인
count = 0
for u, v, data in integrated_graph.edges(data=True):
    if data.get('mode') == 'transfer':
        print(f"환승 엣지: {u} ↔ {v}, time={data['weight']:.2f}분")
        count += 1
        if count >= 10:
            break

환승 엣지: 1호선_도봉산 ↔ 7호선_도봉산, time=3.62분
환승 엣지: 1호선_도봉산 ↔ bus_3640, time=9.45분
환승 엣지: 1호선_도봉산 ↔ bus_3502, time=6.55분
환승 엣지: 1호선_도봉산 ↔ bus_3501, time=9.51분
환승 엣지: 1호선_도봉산 ↔ bus_3653, time=5.07분
환승 엣지: 1호선_도봉산 ↔ bus_3651, time=4.70분
환승 엣지: 1호선_도봉산 ↔ bus_3652, time=4.87분
환승 엣지: 1호선_도봉산 ↔ bus_3503, time=6.46분
환승 엣지: 1호선_도봉산 ↔ bus_3457, time=12.91분
환승 엣지: 1호선_도봉산 ↔ bus_3713, time=14.41분


In [ ]:
import pandas as pd

# 3-1) 노드 정보 추출 & DataFrame 생성
nodes_data = []
for node, data in integrated_graph.nodes(data=True):
    mode = data.get('mode')      # 안전하게 가져오기
    rd = {
        'node_key': node,
        'mode':     mode,
        'lat':      data.get('lat'),
        'lng':      data.get('lng')
    }
    if mode == 'metro':
        rd['station'] = data.get('station')
        rd['line']    = data.get('line')
    elif mode == 'bus':
        rd['stop_idx']= data.get('stop_idx')
        rd['route']   = data.get('route')
    else:
        # mode가 없는(또는 다른) 노드라면 필요에 따라 처리
        rd['note'] = 'unknown mode'
    nodes_data.append(rd)

int_nodes_df = pd.DataFrame(nodes_data)
print(f"총 통합 노드 수: {len(int_nodes_df)}")
display(int_nodes_df.head(10))

# 3-2) 엣지 정보 추출 & DataFrame 생성
edges_data = []
for u, v, data in integrated_graph.edges(data=True):
    edge_info = {
        'from':       u,
        'to':         v,
        'mode':       data.get('mode'),
        'weight_min': data.get('weight')
    }

    # 버스 엣지라면 어떤 노선들이 사용하는지도 저장
    if 'lines' in data:
        edge_info['lines'] = list(data['lines'])  # set → list로 변환

    edges_data.append(edge_info)

int_edges_df = pd.DataFrame(edges_data)
print(f"총 통합 엣지 수: {len(int_edges_df)}")
display(int_edges_df.head(10))


총 통합 노드 수: 36707


,node_key,mode,lat,lng,station,line,stop_idx,route,note
0,1호선_연천,metro,38.100730,127.073720,연천,1호선,NaN,NaN,NaN
1,1호선_전곡,metro,38.024580,127.071800,전곡,1호선,NaN,NaN,NaN
2,1호선_청산,metro,37.981720,127.069120,청산,1호선,NaN,NaN,NaN
3,1호선_소요산,metro,37.948100,127.061034,소요산,1호선,NaN,NaN,NaN
4,1호선_동두천,metro,37.927878,127.054790,동두천,1호선,NaN,NaN,NaN
5,1호선_보산,metro,37.913702,127.057277,보산,1호선,NaN,NaN,NaN
6,1호선_동두천중앙,metro,37.901885,127.056482,동두천중앙,1호선,NaN,NaN,NaN
7,1호선_지행,metro,37.892334,127.055716,지행,1호선,NaN,NaN,NaN
8,1호선_덕정,metro,37.843188,127.061277,덕정,1호선,NaN,NaN,NaN
9,1호선_덕계,metro,37.818486,127.056486,덕계,1호선,NaN,NaN,NaN


총 통합 엣지 수: 55030


,from,to,mode,weight_min,lines
0,1호선_연천,1호선_전곡,metro_edge,0.0,NaN
1,1호선_전곡,1호선_청산,metro_edge,0.0,NaN
2,1호선_청산,1호선_소요산,metro_edge,0.0,NaN
3,1호선_소요산,1호선_동두천,metro_edge,0.0,NaN
4,1호선_동두천,1호선_보산,metro_edge,0.0,NaN
5,1호선_보산,1호선_동두천중앙,metro_edge,0.0,NaN
6,1호선_동두천중앙,1호선_지행,metro_edge,0.0,NaN
7,1호선_지행,1호선_덕정,metro_edge,0.0,NaN
8,1호선_덕정,1호선_덕계,metro_edge,0.0,NaN
9,1호선_덕계,1호선_양주,metro_edge,0.0,NaN


############ 후보지 생성.

In [ ]:
# ——————————————————————————————
#  1) KD-Tree 생성: 위경도 기반 가까운 노드 탐색
# ——————————————————————————————

# (1-1) 위/경도 좌표가 있는 노드만 추출
valid_nodes = []
coords = []
for node, data in integrated_graph.nodes(data=True):
    lat, lng = data.get('lat'), data.get('lng')
    if lat is None or lng is None:
        continue
    valid_nodes.append(node)
    coords.append((lat, lng))

# (1-2) numpy 배열 & KDTree 생성
all_nodes  = valid_nodes
all_coords = np.array(coords)  # shape=(N, 2)
all_tree   = cKDTree(all_coords)


In [ ]:
def nearby_multimodal_nodes(user_lat, user_lon, radius_m=1000):
    radius_deg = radius_m / 111000.0
    idxs = all_tree.query_ball_point([user_lat, user_lon], r=radius_deg)
    print(f"→ radius_deg={radius_deg:.5f}, found {len(idxs)} candidate indices")
    for i in idxs[:10]:
        print("   sample node:", all_nodes[i], "coord:", all_coords[i])
    results = []
    for i in idxs:
        node = all_nodes[i]
        lat2, lon2 = integrated_graph.nodes[node]['lat'], integrated_graph.nodes[node]['lng']
        t_min = walk_time_via_road(user_lat, user_lon, lat2, lon2)
        results.append((node, t_min))
    results.sort(key=lambda x: x[1])
    return results


In [ ]:
# ——————————————————————————————
#  2) 도로망 기반 보행시간 계산
# ——————————————————————————————

def walk_time_via_road(lat1, lon1, lat2, lon2):
    """
    두 지점 간 도로망(shortest-path) 보행시간 계산 (분 단위)
    - 경로가 없으면 허버사인 거리로 fallback
    """
    try:
        # 도로망에서 가장 가까운 노드 매핑
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        # 최단 경로 길이 (미터)
        dist_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        # 네트워크 연결 불가 시 허버사인 거리 사용
        dist_m = haversine(lat1, lon1, lat2, lon2)
    # 거리 → 시간(분) 환산
    return (dist_m / 1000.0) / WALK_SPEED * 60.0


In [ ]:
# 1) 계절 계산 함수
# ——————————————————————————————
def get_current_season(now=None):
    """
    now.month를 기준으로 계절 반환
      3~5월   봄 ('spring')
      6~8월   여름 ('summer')
      9~11월  가을 ('fall')
      12,1,2월 겨울 ('winter')
    """
    if now is None:
        now = datetime.now()
    m = now.month
    if 3 <= m <= 5:
        return 'spring'
    if 6 <= m <= 8:
        return 'summer'
    if 9 <= m <= 11:
        return 'fall'
    return 'winter'

# 모드별 허용 계절(필요시 수정)
SEASON_SERVICE = {
    'metro': ['spring','summer','fall','winter'],
    'bus':   ['spring','summer','fall','winter']
}


In [ ]:
# ──────────────────────────────────────────
# 2) 정교한 운행 시간 로직 함수
# ──────────────────────────────────────────

def is_metro_operating(now=None):
    if now is None:
        now = datetime.now().time()
    # 05:30 ≤ now < 24:00  또는  00:00 ≤ now < 00:30
    return (now >= time(5,30)) or (now < time(0,30))

from zoneinfo import ZoneInfo  # 파이썬 3.9+

def is_bus_operating(node, now=None):
    """
    일반버스: 05:30 <= now <= 23:59:59
    N-야간버스: 00:00 <= now < 05:30
    """
    if now is None:
        # 한국시간 기준으로 now
        now = datetime.now(ZoneInfo("Asia/Seoul")).time()

    # integrated_graph에 저장된 route 속성 가져오기
    raw = integrated_graph.nodes[node]['route']
    # raw가 str이면 [raw], 그 외(리스트 등)면 list(raw)
    lines = [raw] if isinstance(raw, str) else list(raw)

    # 일반버스 리스트, 야간버스 리스트
    general = [r for r in lines if not r.startswith('N')]
    night   = [r for r in lines if     r.startswith('N')]

    # 1) 일반버스 운행 여부
    if general and time(5,30) <= now <= time(23,59,59):
        return True
    # 2) 야간버스 운행 여부
    if night and time(0,0) <= now < time(5,30):
        return True

    return False


In [ ]:
# ——————————————————————————————
# (B) bus_nodes: 'mode'=='bus' AND 'route' 속성이 있는 노드만
# ——————————————————————————————
bus_nodes = [
    n for n,d in integrated_graph.nodes(data=True)
    if d.get('mode')=='bus' and d.get('route') is not None
]

# 헬퍼: node에 걸리는 노선 리스트로 정리
def get_bus_lines(node):
    raw = integrated_graph.nodes[node].get('route') or []
    return [raw] if isinstance(raw, str) else list(raw)

# ——————————————————————————————
# (C) 일반버스 전용 노드 하나
# ——————————————————————————————
gen_node = next(
    n for n in bus_nodes
    if get_bus_lines(n)              # 노선 하나라도 있고
    and all(not r.startswith('N') for r in get_bus_lines(n))  # 모두 일반
)

# ——————————————————————————————
# (D) N-야간버스 전용 노드 하나
# ——————————————————————————————
night_node = next(
    n for n in bus_nodes
    if get_bus_lines(n)
    and all(r.startswith('N') for r in get_bus_lines(n))  # 모두 N-야간
)

# ——————————————————————————————
# (E) 테스트 출력
# ——————————————————————————————
test_times = [
    time(0,0),  time(0,30), time(2,0),
    time(4,0),  time(5,0),  time(5,30),
    time(10,0), time(23,0), time(13,30)
]

print("Time  | Metro | GenBus | NightBus")
for t in test_times:
    m = '●' if is_metro_operating(t) else '×'
    g = '●' if is_bus_operating(gen_node, t) else '×'
    x = '●' if is_bus_operating(night_node, t) else '×'
    print(f"{t:%H:%M} |   {m}   |    {g}   |     {x}")

Time  | Metro | GenBus | NightBus
00:00 |   ●   |    ×   |     ●
00:30 |   ×   |    ×   |     ●
02:00 |   ×   |    ×   |     ●
04:00 |   ×   |    ×   |     ●
05:00 |   ×   |    ×   |     ●
05:30 |   ●   |    ●   |     ×
10:00 |   ●   |    ●   |     ×
23:00 |   ●   |    ●   |     ×
13:30 |   ●   |    ●   |     ×


In [ ]:
def filter_candidates_by_availability(candidates, now=None):
    """
    - candidates: [(node_key, walk_min), …]
    - 반환: 현재 계절·운행 중인 모드만 필터링
    """
    if now is None:
        now = datetime.now().time()
    season = get_current_season()

    print(f"\n[필터 시작] season={season}, now={now:%H:%M:%S}, 후보 수={len(candidates)}")
    valid = []
    for node, walk in candidates:
        mode = integrated_graph.nodes[node].get('mode')
        # 1) 계절 허용 여부
        if season not in SEASON_SERVICE.get(mode, []):
            print(f"  ✗ {node}({mode}) — 계절 불허 ({season})")
            continue

        # 2) 시간대별 운행 여부
        if mode == 'metro':
            ok = is_metro_operating(now)
            print(f"  {'●' if ok else '✗'} {node}(metro) — is_metro_operating? {ok}")
            if not ok:
                continue

        elif mode == 'bus':
            ok = is_bus_operating(node, now)
            print(f"  {'●' if ok else '✗'} {node}(bus)   — is_bus_operating? {ok}")
            if not ok:
                continue

        else:
            # mode 잘못 들어갔을 경우
            print(f"  ⚠ {node} — 알 수 없는 mode={mode}")
            continue

        # 여기까지 통과한 후보만 valid에 추가
        valid.append((node, walk))

    print(f"[필터 완료] 살아남은 후보 수={len(valid)}\n")
    return valid


In [ ]:
# (1) 현재 시각
from datetime import datetime, timedelta

# UTC 기준 +9시간
now = (datetime.utcnow() + timedelta(hours=9)).time()


# (2) KDTree → 도로망 보행시간 계산으로 후보지 생성
start_candidates = nearby_multimodal_nodes(start_lat, start_lon,  radius_m=1000)
end_candidates   = nearby_multimodal_nodes(end_lat,   end_lon,    radius_m=1000)

# (3) 운행·계절 필터 적용
start_candidates = filter_candidates_by_availability(start_candidates, now)
end_candidates   = filter_candidates_by_availability(end_candidates,   now)

# (4) 결과 확인
print("출발 후보 (운행·계절 필터, 상위 10):", start_candidates[:10])
print("도착 후보 (운행·계절 필터, 상위 10):", end_candidates[:10])


→ radius_deg=0.00901, found 280 candidate indices
   sample node: 01B_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 202_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 472_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: N73_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: N30_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 261_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 152_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 100_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 105_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 9401_국가인권위.안중근활동터 coord: [ 37.56435047 126.98785152]
→ radius_deg=0.00901, found 254 candidate indices
   sample node: 서초03_삼성레포츠 coord: [ 37.49220643 127.02081643]
   sample node: 서초03_삼성래미안아파트 coord: [ 37.49496599 127.01849311]
   sample node: 서초10_삼성래미안아파트 coord: [ 37.49496599 127.01849311]
   sample node: 서초21_서초2동주민센터.서이초등학교 coord: [

In [ ]:
def dedup_bus_by_route_nearest(candidates):
    """
    동일한 버스노선(route) 중 사용자 위치에서 가장 가까운 하나만 유지
    - metro는 그대로 유지
    """
    best_bus = {}  # key: route → (node, walk)
    final = []

    for node, walk in candidates:
        data = integrated_graph.nodes[node]
        mode = data.get('mode')

        if mode == 'bus':
            route = data.get('route')
            if route not in best_bus or walk < best_bus[route][1]:
                best_bus[route] = (node, walk)
        else:
            final.append((node, walk))  # 지하철은 그대로 유지

    # 가장 가까운 정류장만 남기기
    final.extend(best_bus.values())
    return final


In [ ]:
start_candidates = dedup_bus_by_route_nearest(start_candidates)
end_candidates   = dedup_bus_by_route_nearest(end_candidates)

print("\n▶ Dedup된 출발 후보:", start_candidates)
print("▶ Dedup된 도착 후보:",   end_candidates)



▶ Dedup된 출발 후보: [('1호선_종로3가', np.float64(0.017749571362737106)), ('3호선_종로3가', np.float64(1.2952134738131797)), ('5호선_종로3가', np.float64(4.194358523874303)), ('5호선_을지로4가', np.float64(10.049091412857074)), ('2호선_을지로4가', np.float64(11.01495205389755)), ('3호선_을지로3가', np.float64(13.302344320797138)), ('2호선_을지로3가', np.float64(14.209931287710392)), ('종로12_종로3가역9번출구', np.float64(0.5231744087494039)), ('273_종로4가.종묘', np.float64(2.831018704562441)), ('720_종로4가.종묘', np.float64(2.831018704562441)), ('140_종로4가.종묘', np.float64(2.831018704562441)), ('7212_종로4가.종묘', np.float64(2.831018704562441)), ('721_종로4가.종묘', np.float64(2.831018704562441)), ('270_종로4가.종묘', np.float64(2.831018704562441)), ('160_종로4가.종묘', np.float64(3.317118151357348)), ('8101_종로4가.종묘', np.float64(3.317118151357348)), ('201_종로4가.종묘', np.float64(3.317118151357348)), ('260_종로4가.종묘', np.float64(3.317118151357348)), ('370_종로4가.종묘', np.float64(3.317118151357348)), ('262_종로4가.종묘', np.float64(3.317118151357348)), ('150_종로4가.종묘', np.float64

# 최단시간 함수.

In [ ]:
def get_fastest_routes_optimized(G, origin_candidates, dest_candidates,
                                 k=10, max_transfers=4):
    """
    G: 통합 그래프 (metro+bus+transfer)
    origin_candidates: [(origin_node, origin_walk_min), …]
    dest_candidates:   [(dest_node,   dest_walk_min),   …]
    k: 상위 k개 결과 반환
    max_transfers: 허용할 최대 환승 횟수
    """
    fastest_list = []

    for origin_node, origin_walk in origin_candidates:
        # 1) 한 번만 Dijkstra
        distances, paths = nx.single_source_dijkstra(
            G, origin_node, weight='weight'
        )

        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue

            path        = paths[dest_node]
            travel_time = distances[dest_node]
            total_time  = travel_time + origin_walk + dest_walk

            # ——— 시작/끝 “transfer” 엣지 제외 ———
            if len(path) >= 2:
                # 시작
                if G[path[0]][path[1]].get('mode') == 'transfer':
                    continue
                # 끝
                if G[path[-2]][path[-1]].get('mode') == 'transfer':
                    continue

            # 2) 환승 횟수 계산
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(path, path[1:]))

            for idx, (u, v) in enumerate(edges):
                m = G[u][v].get('mode')
                # 중간 환승 엣지
                if m == 'transfer':
                    if 0 < idx < len(edges)-1:
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                # metro_edge: 호선 변경 시
                if m == 'metro_edge':
                    cur_line = G.nodes[u]['line']
                    if prev_mode=='metro_edge' and prev_line!=cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                # bus_edge: 노선 변경 시
                if m == 'bus_edge':
                    cur_route = next(iter(G[u][v].get('lines', [])), None)
                    if prev_mode=='bus_edge' and prev_line!=cur_route:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_route
                    continue
                prev_mode, prev_line = m, None

            # 3) 환승 제한
            if transfer_count > max_transfers:
                continue

            fastest_list.append({
                'path':      path,
                'time_min':  total_time,
                'transfers': transfer_count,
                'walk_min':  origin_walk + dest_walk
            })

    # 상위 k개만
    fastest_list.sort(key=lambda x: x['time_min'])
    return fastest_list[:k]


출발지, 도착지 관련 문제가 아직 남아있음.

In [ ]:
# 6) 최단 시간 경로 뽑기
routes = get_fastest_routes_optimized(
    integrated_graph,
    start_candidates,
    end_candidates,
    k=20,
    max_transfers=4
)

# 7) 결과 출력
for i, r in enumerate(routes, 1):
    print(f"\n{i}위: 총시간={r['time_min']:.1f}분 "
          f"(보행={r['walk_min']:.1f}분, 환승={r['transfers']}회)")
    print("  →", " → ".join(r['path']))



1위: 총시간=28.4분 (보행=2.9분, 환승=1회)
  → 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남

2위: 총시간=36.8분 (보행=12.4분, 환승=1회)
  → 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현

3위: 총시간=38.3분 (보행=14.9분, 환승=1회)
  → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남

4위: 총시간=39.7분 (보행=5.1분, 환승=2회)
  → 5호선_종로3가 → 5호선_을지로4가 → 5호선_동대문역사문화공원 → 5호선_청구 → 5호선_신금호 → 5호선_행당 → 5호선_왕십리 → 수인분당선_왕십리 → 수인분당선_서울숲 → 수인분당선_압구정로데오 → 수인분당선_강남구청 → 수인분당선_선정릉 → 수인분당선_선릉 → 2호선_선릉 → 2호선_역삼 → 2호선_강남

5위: 총시간=41.8분 (보행=11.6분, 환승=3회)
  → 5호선_을지로4가 → 5호선_동대문역사문화공원 → 5호선_청구 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남

6위: 총시간=41.9분 (보행=12.6분, 환승=2회)
  → 2호선_을지로4가 → 2호선_을지로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선

# 환승편의성

In [ ]:
import json
import re
import pandas as pd
import numpy as np
import networkx as nx
from geopy.distance import geodesic
from itertools import islice

# —————————————————————————
# 1) 데이터 로드 & 매핑표 생성
# —————————————————————————

# 1.1) 환승보행시간 데이터 로드
df_tt = pd.read_json('metro_transfer_time .json')
df_tt = (
    df_tt
    .rename(columns={
        '호선':'line',
        '환승역명':'station',
        '환승노선':'transfer_line',
        '환승소요시간':'duration_str'
    })
    # '1호선' 형태로 맞추고 "MM:SS" → 분 단위 float
    .assign(
        line=lambda df: df['line'].astype(str)+'호선',
        duration_min=lambda df: df['duration_str']
            .str.split(':')
            .apply(lambda t: int(t[0]) + int(t[1])/60)
    )
)
transfer_time_map = {
    (row.line, row.transfer_line, row.station): row.duration_min
    for _, row in df_tt.iterrows()
}

# 1.2) 지하철 내부 정보 로드 (에스컬레이터 길이)
with open('metro_inside.json','r',encoding='utf-8') as f:
    df_ins = pd.DataFrame(json.load(f))
# 괄호 제거된 역명 컬럼 추가
df_ins['station_clean'] = df_ins['station_name'].str.replace(r'\(.*\)$','',regex=True)
es_map = {
    (row['lines'], row['station_clean']): row['escalator_optimal_length']
    for _, row in df_ins.iterrows()
}

# 1.2.3) 지면 고도 맵: (호선, 역명) → ground_elevation (m)
ge_map = {
    (row['lines'], row['station_clean']): row['ground_elevation']
    for _, row in df_ins.iterrows()
}

# 보행·에스컬레이터 속도 정의 (m/min)
WALK_SPEED = 4000 / 60   # 4km/h → 약 66.7 m/min
ES_SPEED   = 8000 / 60   # 8km/h → 약133.3 m/min


In [ ]:
def calc_escalator_time(line, station, es_map):
    """
    에스컬레이터 길이 기반 환승 시간(분)
    """
    es_len = es_map.get((line, station), 0)
    if es_len > 0:
        return es_len / ES_SPEED  # 분 단위
    return 0

def calc_transfer_walk_time(path, G, transfer_time_map, es_map):
    """
    경로 내 지하철 환승 엣지마다 환승 소요시간 또는 에스컬레이터 환승시간 적용
    """
    transfer_time_sum = 0
    edges = list(zip(path, path[1:]))
    for idx, (u, v) in enumerate(edges):
        attr = G[u][v]
        if attr.get('mode') == 'transfer':
            # 지하철-지하철 환승인지 체크
            u_mode, v_mode = G.nodes[u].get('mode'), G.nodes[v].get('mode')
            u_line, v_line = G.nodes[u].get('line'), G.nodes[v].get('line')
            u_station, v_station = G.nodes[u].get('station'), G.nodes[v].get('station')

            # 둘 다 지하철 & 호선 다르면 환승
            if u_mode=='metro' and v_mode=='metro' and u_line!=v_line:
                # transfer_time_map에서 환승시간 있으면
                key = (u_line, v_line, u_station)
                t = transfer_time_map.get(key)
                if t is None:
                    # 반대 방향도 체크
                    key_rev = (v_line, u_line, v_station)
                    t = transfer_time_map.get(key_rev)
                if t is not None:
                    transfer_time_sum += t
                else:
                    # 환승 소요시간 정보 없으면 에스컬레이터 길이 사용
                    t_es = calc_escalator_time(u_line, u_station, es_map)
                    transfer_time_sum += t_es
    return transfer_time_sum


In [ ]:
def get_transfer_convenient_routes(G, origin_candidates, dest_candidates,
                                   transfer_time_map, es_map,
                                   k=10, max_transfers=4,
                                   weight_transfer=3.0,
                                   weight_walk=1.0,
                                   weight_internal=1.5):
    def calc_transfer_score(route, origin_walk, dest_walk):
        transfer_count = 0
        transfer_internal_time = 0.0
        edges = list(zip(route, route[1:]))
        prev_mode, prev_line = None, None

        for idx, (u, v) in enumerate(edges):
            mode = G[u][v].get('mode')
            if mode == 'transfer':
                u_mode = G.nodes[u].get('mode')
                v_mode = G.nodes[v].get('mode')
                u_line = G.nodes[u].get('line')
                v_line = G.nodes[v].get('line')
                u_station = G.nodes[u].get('station')
                v_station = G.nodes[v].get('station')

                if u_mode == 'metro' and v_mode == 'metro' and u_line != v_line:
                    key = (u_line, v_line, u_station)
                    rev_key = (v_line, u_line, v_station)
                    t = transfer_time_map.get(key) or transfer_time_map.get(rev_key)
                    if t is None:
                        t = calc_escalator_time(u_line, u_station, es_map)
                    transfer_internal_time += t
                    if 0 < idx < len(edges) - 1:
                        transfer_count += 1
            elif mode == 'metro_edge':
                cur_line = G.nodes[u].get('line')
                if prev_mode == 'metro_edge' and prev_line != cur_line:
                    transfer_count += 1
                prev_mode, prev_line = 'metro_edge', cur_line
            elif mode == 'bus_edge':
                cur_route = next(iter(G[u][v].get('lines', [])), None)
                if prev_mode == 'bus_edge' and prev_line != cur_route:
                    transfer_count += 1
                prev_mode, prev_line = 'bus_edge', cur_route
            else:
                prev_mode, prev_line = mode, None

        total_walk = origin_walk + dest_walk
        score = (
            transfer_count * weight_transfer +
            transfer_internal_time * weight_internal +
            total_walk * weight_walk
        )
        return score, transfer_count, total_walk, transfer_internal_time

    results = []

    for origin_node, origin_walk in origin_candidates:
        distances, paths = nx.single_source_dijkstra(G, origin_node, weight='weight')
        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue
            path = paths[dest_node]
            score, trans_cnt, ext_walk, internal_walk = calc_transfer_score(
                path, origin_walk, dest_walk
            )
            if trans_cnt > max_transfers:
                continue
            results.append({
                'path': path,
                'score': score,
                'transfers': trans_cnt,
                'walk_min': ext_walk,
                'internal_walk_min': internal_walk
            })

    results.sort(key=lambda x: x['score'])
    return results[:k]


In [ ]:
routes = get_transfer_convenient_routes(
    integrated_graph, start_candidates, end_candidates,
    transfer_time_map, es_map,
    k=10, max_transfers=4
)
for i, r in enumerate(routes, 1):
    print(f"\n[{i}] 환승편의성={r['score']:.1f} "
          f"(환승={r['transfers']}회, 외부도보={r['walk_min']:.1f}분, 환승도보={r['internal_walk_min']:.1f}분)")
    print("  경로:", " → ".join(r['path']))



[1] 환승편의성=9.5 (환승=0회, 외부도보=9.5분, 환승도보=0.0분)
  경로: 140_종로4가.종묘 → 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[2] 환승편의성=10.5 (환승=2회, 외부도보=2.9분, 환승도보=1.1분)
  경로: 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남

[3] 환승편의성=11.6 (환승=2회, 외부도보=1.6분, 환승도보=2.7분)
  경로: 1호선_종로3가 → 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남

[4] 환승편의성=13.5 (환승=0회, 외부도보=13.5분, 환승도보=0.0분)
  경로: 470_종로2가.삼일교 → 470_종로2가 → 470_종로2가사거리 → 470_국가인권위.안중근활동터 → 470_순천향대학병원 → 470_한남오거리 → 470_한남대교전망카페 → 470_신사역 → 470_논현역 → 470_지하철2호선강남역

[5] 환승편의성=13.5 (환승=0회, 외부도보=13.5분, 환승도보=0.0분)
  경로: 741_종로2가.삼일교 → 741_종로2가 → 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[6] 환승편의성=13.6 (환승=0회, 외부도보=13.6분, 환승도보=0.0분)
  경로: 140_종로3가.탑골공원 → 140_국가인권위

In [ ]:
# 0. Colab 파일 업로드 (한 번만 필요)
uploaded = files.upload()

Saving bus_confuse.csv to bus_confuse.csv
Saving metro_confuse.xlsx to metro_confuse (1).xlsx
Saving bus_speed.xlsx to bus_speed (1).xlsx
Saving metro_late.xlsx to metro_late (1).xlsx
Saving bus_late.xlsx to bus_late (1).xlsx
Saving bus_accident.xlsx to bus_accident (1).xlsx
Saving metro_accident.json to metro_accident (1).json
Saving weather.xlsx to weather (1).xlsx
Saving metro_dust.xlsx to metro_dust (1).xlsx


#정시성 함수 만들기.
- 사고는 최근 5년, 늦은건 1년 기준
- 클수록 안좋음.

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

# 계절 자동 계산 함수
def get_current_season():
    month = datetime.now(ZoneInfo("Asia/Seoul")).month
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'fall'
    else:
        return 'winter'

# 현재 계절 확인
get_current_season()

'summer'

In [ ]:
# 엑셀 파일
df_bus_late = pd.read_excel('bus_late.xlsx', dtype=str)
df_bus_acc = pd.read_excel('bus_accident.xlsx', dtype=str)
df_metro_late = pd.read_excel('metro_late.xlsx')
with open('metro_accident.json', encoding='utf-8') as f:
    metro_accidents = {row['역명']: row['사고횟수'] for row in json.load(f)}


In [ ]:
for df in [df_bus_late, df_bus_acc]:
    if '정류소명' in df.columns:
        df['정류소명'] = df['정류소명'].str.replace('\ufeff', '', regex=False)

df_bus_late['평균2분이하건수'] = pd.to_numeric(df_bus_late['평균2분이하건수'], errors='coerce').fillna(0).astype(int)
df_bus_late['평균2분초과건수'] = pd.to_numeric(df_bus_late['평균2분초과건수'], errors='coerce').fillna(0).astype(int)


In [ ]:
bus_stops_df = pd.read_excel('bus_stops.xlsx')

In [ ]:
# bus_stops.xlsx 또는 해당 DataFrame에서
# 실제 컬럼명은 'NODE_ID', 'ARS_ID', '정류소명', 'X좌표', 'Y좌표'
STOP_IDX_TO_NAME = dict(zip(bus_stops_df['NODE_ID'], bus_stops_df['정류소명']))


In [ ]:
# ──────────────────────────────────────────────
# 2. 유틸 함수들
# ──────────────────────────────────────────────

def clean_stop_name(s):
    return str(s).replace(' ','').replace('.', '').replace('·','').replace('(','').replace(')','').replace('-','').strip()

def clean_route_num(s):
    return str(s).replace('\ufeff', '').strip()

def get_bus_late(data):
    node_key = data.get('node_key')
    route = str(data.get('route'))
    if not node_key or not route:
        return 0, 0

    stop_name = node_key.split("_", 1)[1]
    stop_name_clean = clean_stop_name(stop_name)
    route_clean = clean_route_num(route)

    row = df_bus_late[
        (df_bus_late['정류소명_clean'] == stop_name_clean) &
        (df_bus_late['버스번호_clean'] == route_clean)
    ]
    if row.empty:
        return 0, 0

    n1 = int(row['평균2분이하건수'].values[0])
    n2 = int(row['평균2분초과건수'].values[0])
    total = n1 + n2 if (n1 + n2) > 0 else 1
    delay_prob = n2 / total
    if n2 >= 3 * n1:
        t = 2.7
    elif n2 >= 2 * n1:
        t = 2.3
    elif n2 >= n1:
        t = 2.0
    else:
        t = 3.0
    return t, delay_prob

def get_bus_accident_prob(data):
    route = str(data.get('route'))
    if not route:
        return 0
    route_clean = clean_route_num(route)
    row = df_bus_acc[df_bus_acc['버스번호_clean'] == route_clean]
    cnt = int(float(row['사고횟수'].values[0])) if not row.empty else 0
    return cnt / (5 * 365)

def get_metro_late(df_metro_late, station, line, season):
    df = df_metro_late[(df_metro_late['line'] == line) & (df_metro_late['season'] == season)]
    if df.empty:
        return 0.0, 0.0
    delay_time = df['지연시간'].mean()
    delay_prob = 1.0  # 고정값
    return delay_time, delay_prob

def get_metro_accident_prob(metro_accidents, station):
    cnt = metro_accidents.get(station, 0)
    return cnt / (5 * 365)

def calc_route_punctuality(route, G, season, df_metro_late, metro_accidents):
    metro_delay_scores = []
    metro_accident_scores = []
    bus_scores = []

    seen_metro_lines = set()

    for node in route:
        data = G.nodes[node]
        mode = data.get('mode')

        if mode == 'metro':
            line = data.get('line')
            station = data.get('station')

            # ⏱️ (1) 지연 점수 (호선당 1번만)
            if line not in seen_metro_lines:
                delay_t, delay_p = get_metro_late(df_metro_late, station, line, season)
                metro_delay_scores.append(delay_p * delay_t)
                seen_metro_lines.add(line)

            # ⚠ (2) 사고 점수 (역별 모두 누적)
            accident_p = get_metro_accident_prob(metro_accidents, station)
            metro_accident_scores.append(accident_p)

        elif mode == 'bus':
            t, p = get_bus_late(data)
            accident_p = get_bus_accident_prob(data)
            score = (p * t) + accident_p
            bus_scores.append(score)

    # 📊 평균 처리
    avg_metro_delay = np.mean(metro_delay_scores) if metro_delay_scores else 0
    avg_metro_accident = np.mean(metro_accident_scores) if metro_accident_scores else 0
    total_bus_score = sum(bus_scores)

    # ✅ 지하철 점수는 1/40로 축소
    subway_score = (avg_metro_delay + avg_metro_accident) / 100

    return subway_score + total_bus_score



def get_best_punctuality_routes(G, origin_candidates, dest_candidates, df_metro_late, metro_accidents,
                                k=10, max_transfers=4):
    season = get_current_season()
    results = []
    for origin_node, origin_walk in origin_candidates:
        distances, paths = nx.single_source_dijkstra(G, origin_node, weight='weight')
        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue
            route = paths[dest_node]
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(route, route[1:]))
            for idx, (u, v) in enumerate(edges):
                m = G[u][v].get('mode')
                if m == 'transfer':
                    if 0 < idx < len(edges)-1:
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                if m == 'metro_edge':
                    cur_line = G.nodes[u].get('line')
                    if prev_mode == 'metro_edge' and prev_line != cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                if m == 'bus_edge':
                    cur_route = G.nodes[u].get('route')
                    if prev_mode == 'bus_edge' and prev_line != cur_route:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_route
                    continue
                prev_mode, prev_line = m, None
            if transfer_count > max_transfers:
                continue

            punctuality_score = calc_route_punctuality(route, G, season, df_metro_late, metro_accidents)
            results.append({
                'path': route,
                'score': punctuality_score,
                'transfers': transfer_count,
                'walk_min': origin_walk + dest_walk
            })
    results.sort(key=lambda x: x['score'])
    return results[:k]

In [ ]:
routes = get_best_punctuality_routes(
    integrated_graph,
    start_candidates,
    end_candidates,
    df_metro_late,
    metro_accidents,
    k=100,
    max_transfers=4
)

for i, r in enumerate(routes, 1):
    print(f"\n[{i}] 정시성점수={r['score']:.5f} (환승={r['transfers']}회, 도보={r['walk_min']:.1f}분)")
    print("  경로:", " → ".join(r['path']))



[1] 정시성점수=0.00767 (환승=0회, 도보=27.6분)
  경로: 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[2] 정시성점수=0.00877 (환승=0회, 도보=13.6분)
  경로: 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[3] 정시성점수=0.00877 (환승=0회, 도보=20.2분)
  경로: 140_중구청앞.덕수중학교 → 140_충무로역8번출구 → 140_충무로역.한옥마을.한국의집 → 140_남산1호터널 → 140_순천향대학병원.한남오거리 → 140_신사역.푸른저축은행 → 140_논현역 → 140_지하철2호선강남역

[4] 정시성점수=0.00877 (환승=0회, 도보=31.9분)
  경로: 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역 → 140_신분당선강남역

[5] 정시성점수=0.00986 (환승=0회, 도보=9.5분)
  경로: 140_종로4가.종묘 → 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[6] 정시성점수=0.00986 (환승=0회, 도보=17.9분)
  경로: 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역 → 140_신분당선강남역

[7] 정시성점수=0.00986 (환승=0회, 도보=14.5분)
  경로: 140_광장시장 → 14

# 쾌적도 함수

In [ ]:
# 0) 쾌적 점수 매핑 함수 정의 (반드시 이 셀 맨 위에 위치)
def score_microdust_cat(x):
    if x <= 30:
        return 4
    elif x <= 80:
        return 3
    elif x <= 150:
        return 2
    else:
        return 1

def score_temp_cat(x):
    if 10 <= x <= 24:
        return 4
    elif (0 <= x < 10) or (24 < x <= 28):
        return 3
    elif (-10 <= x < 0) or (28 < x <= 34):
        return 2
    else:
        return 1

def score_humidity_cat(h):
    if 50 <= h <= 60:
        return 4
    elif 40 <= h < 50:
        return 3
    elif h < 40:
        return 2
    else:
        return 1

# 불쾌지수 = 0.81 * T + 0.01 * RH * (0.99 * T - 14.3) + 46.3
def discomfort_index(temp, humidity):
    return 0.81 * temp + 0.01 * humidity * (0.99 * temp - 14.3) + 46.3

# 불쾌지수 점수 매핑 (범위 기반)
def score_discomfort(idx):
    if idx < 68:
        return 4
    elif idx < 75:
        return 3
    elif idx < 80:
        return 2
    else:
        return 1


In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# —— 1) 계절 가져오기 ————————————————————————————————
current_season = get_current_season()  # 'spring','summer','fall','winter'

# —— 2) 환경 지표 로드 & 필터링 ——————————————————————————
# metro_dust.xlsx: ['name','line','미세먼지','계절']
dust_df   = pd.read_excel('metro_dust.xlsx', dtype={'name':str, 'line':str, '미세먼지':float, '계절':str})
dust_season = dust_df[dust_df['season'] == current_season].copy()

# weather.xlsx: ['지역','season','avg_humidity','avg_temp','lng','lat','평균미세먼지']
weather_df = pd.read_excel(
    'weather.xlsx',
    dtype={'season': str},
    converters={
        'avg_temp':      lambda x: float(str(x).lstrip('\ufeff')),
        'avg_humidity':  lambda x: float(str(x).lstrip('\ufeff')),
        'lat':           lambda x: float(str(x).lstrip('\ufeff')),
        'lng':           lambda x: float(str(x).lstrip('\ufeff'))
    }
)


weather_season = weather_df[weather_df['season'] == current_season].copy()

# —— 3) 매핑 테이블 & KDTree 준비 ————————————————————————
# 3-1) dust_map: station_name → score(1~4)
dust_season['dust_score'] = dust_season['미세먼지'].apply(score_microdust_cat)
dust_map = dict(zip(dust_season['name'], dust_season['dust_score']))

# 3-2) weather KDTree: (lat,lng) → (temp_score, humidity_score)
weather_season['temp_score']     = weather_season['avg_temp'].apply(score_temp_cat)
weather_season['humid_score']    = weather_season['avg_humidity'].apply(score_humidity_cat)
coords_weather = weather_season[['lat','lng']].to_numpy()
weather_tree   = cKDTree(coords_weather)
weather_records = weather_season.reset_index(drop=True)


In [ ]:
weather_records

,지역,season,avg_humidity,avg_temp,lng,lat,평균실외미세먼지,temp_score,humid_score
0,강남구,summer,77.45,25.055699,127.046412,37.512208,23.133761,3,1
1,강동구,summer,82.01,24.939541,127.146303,37.554706,22.029731,3,1
2,강북구,summer,77.38,25.442504,126.999613,37.636093,24.797779,3,1
3,강서구,summer,78.24,24.276602,126.847931,37.552802,23.542180,3,1
4,관악구,summer,73.92,23.372184,126.950219,37.452863,25.577400,4,1
5,광진구,summer,74.39,25.102552,127.077008,37.540704,22.888077,3,1
6,구로구,summer,73.46,24.229555,126.831229,37.486113,19.221661,3,1
7,금천구,summer,73.38,24.203187,126.914121,37.460314,21.026524,3,1
8,노원구,summer,74.21,24.009665,127.087304,37.622196,22.341908,3,1
9,도봉구,summer,77.95,24.251172,127.033110,37.666091,18.922018,3,1


In [ ]:
def get_weather_scores(lat, lng):
    """가장 가까운 weather 레코드 찾아서 (temp_score, humid_score) 반환"""
    _, idx = weather_tree.query([lat, lng])
    rec = weather_records.loc[idx]
    return rec['temp_score'], rec['humid_score']

# —— 4) 구간별 쾌적 점수 함수 —————————————————————————
def edge_comfort(u, v, G):
    mode = G[u][v].get('mode')
    y, x = G.nodes[u]['lat'], G.nodes[u]['lng']

    # 가장 가까운 날씨 레코드 가져오기
    _, idx = weather_tree.query([y, x])
    rec = weather_records.loc[idx]

    discomfort = discomfort_index(rec['avg_temp'], rec['avg_humidity'])
    dis_score = score_discomfort(discomfort)

    if mode == 'metro_edge':
        st = G.nodes[u]['station']
        dust_score = dust_map.get(st, 2)
    elif mode == 'bus_edge':
        dust_score = score_microdust_cat(rec['평균실외미세먼지'])
    elif mode == 'transfer':
        return 1.0  # 환승은 여전히 낮은 점수
    else:
        return 2.0  # 기타는 중간값

    # 최종 쾌적도 점수: 불쾌지수 70% + 미세먼지 30%
    return round(dis_score * 0.7 + dust_score * 0.3, 2)


# —— 5) 경로 쾌적도 평가 함수 ———————————————————————
def evaluate_route_pleasant(path, G):
    scores = [edge_comfort(u, v, G) for u, v in zip(path, path[1:])]
    return np.mean(scores)

# —— 6) 최적 쾌적 경로 추천 ———————————————————————
def find_most_pleasant_routes(G, origin_cand, dest_cand, k=5, max_transfers=4):
    # (1) 시간 기준 상위 N개 경로 우선 뽑기
    time_routes = get_fastest_routes_optimized(G, origin_cand, dest_cand,
                                              k=50, max_transfers=max_transfers)
    # (2) 각 경로에 쾌적 점수 매핑
    for r in time_routes:
        r['pleasant_score'] = evaluate_route_pleasant(r['path'], G)
    # (3) 쾌적 점수 내림차순 정렬 (높을수록 더 쾌적)
    pleasant = sorted(time_routes,
                      key=lambda x: x['pleasant_score'],
                      reverse=True)
    return pleasant[:k]

# —— 7) 사용 예시 ——————————————————————————————
pleasant_routes = find_most_pleasant_routes(
    integrated_graph, start_candidates, end_candidates, k=100
)

for i, r in enumerate(pleasant_routes, 1):
    print(f"\n[{i}위] 쾌적점수={r['pleasant_score']:.5f}, "
          f"총시간={r['time_min']:.1f}분 (보행={r['walk_min']:.1f}분, 환승={r['transfers']}회)")
    print("경로:", " → ".join(r['path']))


[1위] 쾌적점수=3.30000, 총시간=46.6분 (보행=13.6분, 환승=0회)
경로: 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[2위] 쾌적점수=3.30000, 총시간=48.3분 (보행=16.1분, 환승=0회)
경로: 470_종로2가사거리 → 470_국가인권위.안중근활동터 → 470_순천향대학병원 → 470_한남오거리 → 470_한남대교전망카페 → 470_신사역 → 470_논현역 → 470_지하철2호선강남역

[3위] 쾌적점수=3.30000, 총시간=48.3분 (보행=16.1분, 환승=0회)
경로: 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[4위] 쾌적점수=3.30000, 총시간=57.9분 (보행=27.6분, 환승=0회)
경로: 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[5위] 쾌적점수=3.30000, 총시간=57.9분 (보행=27.6분, 환승=0회)
경로: 470_국가인권위.안중근활동터 → 470_순천향대학병원 → 470_한남오거리 → 470_한남대교전망카페 → 470_신사역 → 470_논현역 → 470_지하철2호선강남역

[6위] 쾌적점수=3.30000, 총시간=57.9분 (보행=27.6분, 환승=0회)
경로: 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[7위] 쾌적점수=3.30000, 총시간=64.9분 (보행=20.2분, 환승=0회)
경로: 140_중구청앞.덕수중학교 → 140_충무로역8번출

# 혼잡도 함수

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from zoneinfo import ZoneInfo

# ————————————————————————————————————————————————————————
# 0. 계절 정보 가져오기
# ————————————————————————————————————————————————————————
def get_current_season():
    month = datetime.now(ZoneInfo("Asia/Seoul")).month
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'fall'
    else:
        return 'winter'

current_season = get_current_season()

In [ ]:
import pandas as pd

# 데이터 경로
metro_confuse_path = 'metro_confuse.xlsx'
bus_confuse_path = 'bus_confuse.csv'
bus_speed_path = 'bus_speed.xlsx'

# 데이터 로드
df_metro_confuse = pd.read_excel(metro_confuse_path)
df_bus_confuse = pd.read_csv(bus_confuse_path)
df_bus_speed = pd.read_excel(bus_speed_path)

# 각 데이터의 컬럼 확인
metro_cols = df_metro_confuse.columns.tolist()
bus_cols = df_bus_confuse.columns.tolist()
speed_cols = df_bus_speed.columns.tolist()

In [ ]:
df_metro_confuse

,요일구분,호선,출발역,5시,6시,7시,8시,9시,10시,11시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,00시
0,일요일,1호선,동대문,5.25,6.350,8.725,13.900,22.625,25.675,27.125,...,35.625,31.450,29.875,25.300,21.125,18.650,18.225,14.500,12.150,5.750
1,일요일,1호선,동묘앞,5.50,6.400,8.700,13.175,21.250,24.150,24.475,...,33.025,30.825,28.150,23.425,20.150,17.650,16.450,13.100,10.400,6.750
2,일요일,1호선,서울,4.15,6.300,7.575,14.050,23.000,28.900,30.125,...,36.625,34.000,31.900,27.875,25.875,23.750,21.825,17.625,11.850,4.450
3,일요일,1호선,시청,4.50,6.525,8.125,12.900,21.600,25.850,26.600,...,34.950,32.475,30.575,25.925,24.575,20.775,19.475,15.225,11.550,6.800
4,일요일,1호선,신설동,6.30,7.100,8.275,13.575,19.425,23.150,21.975,...,30.500,26.375,27.600,22.450,19.125,17.350,15.050,14.975,10.500,6.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,평일,우이신설선,솔밭공원,21.55,17.150,26.325,33.050,35.000,31.600,41.000,...,34.975,46.650,42.950,43.625,30.650,31.050,33.975,26.925,21.400,8.125
1496,평일,우이신설선,솔샘,22.30,16.575,22.575,34.575,30.750,33.300,29.550,...,40.375,47.400,45.450,42.125,36.500,31.075,34.775,29.650,16.725,4.825
1497,평일,우이신설선,신설동,14.35,15.650,27.175,33.425,38.400,30.650,32.675,...,38.250,42.025,45.150,48.175,33.275,32.450,29.900,29.350,18.600,4.350
1498,평일,우이신설선,정릉,21.65,12.075,29.150,37.150,35.200,26.275,35.600,...,34.875,40.325,44.875,51.050,27.450,28.525,33.325,26.225,21.000,5.275


In [ ]:
df_bus_confuse

,season,버스번호,정류소명,00시총승객수,01시총승객수,02시총승객수,03시총승객수,04시총승객수,05시총승객수,06시총승객수,...,14시총승객수,15시총승객수,16시총승객수,17시총승객수,18시총승객수,19시총승객수,20시총승객수,21시총승객수,22시총승객수,23시총승객수
0,fall,0017,남이장군사당,12.109890,12.109890,0.0,0.0,0.0,0.208791,0.445055,...,2.478022,3.384615,4.384615,5.357143,7.274725,8.489011,9.335165,10.494505,11.461538,12.087912
1,fall,0017,산천동,53.835165,53.835165,0.0,0.0,0.0,1.043956,2.236264,...,28.989011,32.274725,36.379121,40.598901,44.225275,47.582418,49.670330,51.197802,52.818681,53.708791
2,fall,0017,산천동리버힐삼성아파트,30.252747,30.252747,0.0,0.0,0.0,0.241758,1.230769,...,2.692308,4.670330,7.252747,10.901099,15.505495,20.087912,23.373626,25.857143,28.395604,30.000000
3,fall,0017,삼성테마트,8.593407,8.593407,0.0,0.0,0.0,0.000000,0.054945,...,0.648352,1.494505,2.384615,3.637363,4.373626,5.659341,6.659341,7.494505,8.098901,8.527473
4,fall,0017,새마을금고,37.225275,37.225275,0.0,0.0,0.0,0.307692,0.340659,...,18.027473,20.873626,24.247253,27.895604,32.032967,34.670330,35.928571,36.346154,36.763736,37.148352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116809,winter,청와대A01,국립고궁박물관,1.366667,1.366667,0.0,0.0,0.0,0.000000,0.000000,...,0.900000,1.144444,1.366667,1.366667,1.366667,1.366667,1.366667,1.366667,1.366667,1.366667
116810,winter,청와대A01,청와대,3.444444,3.444444,0.0,0.0,0.0,0.000000,0.000000,...,2.566667,3.433333,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444,3.444444
116811,winter,청와대A01,춘추문,0.666667,0.666667,0.0,0.0,0.0,0.000000,0.000000,...,0.900000,0.855556,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667
116812,winter,청와대A01,효자로입구,5.711111,5.711111,0.0,0.0,0.0,0.000000,0.000000,...,4.566667,5.711111,5.711111,5.711111,5.711111,5.711111,5.711111,5.711111,5.711111,5.711111


In [ ]:
# lat, lng 컬럼을 float으로 변환
df_bus_speed['lat'] = pd.to_numeric(df_bus_speed['lat'], errors='coerce')
df_bus_speed['lng'] = pd.to_numeric(df_bus_speed['lng'], errors='coerce')

# 다시 속도 점수 계산
for col in seasonal_cols:
    df_bus_speed[f'{col}_score'] = BUS_SPEED_REF / df_bus_speed[col]

# 시즌별 버스 속도 보정 매핑 생성
bus_speed_map = {
    season: {
        (round(row['lat'], 5), round(row['lng'], 5)): row[f'{season}_score']
        for _, row in df_bus_speed.iterrows()
    }
    for season in seasonal_cols
}

# 결과 확인
list(bus_speed_map.keys())


['winter', 'spring', 'summer', 'fall']

In [ ]:
import numpy as np

# ————————————————————————————————————————————————————————
# 2. 지하철 혼잡도 전처리
# ————————————————————————————————————————————————————————
# 동일 (호선, 출발역)에 대해 05~00시 전체 시간대 평균을 구함
df_metro_avg = (
    df_metro_confuse
    .drop(columns=['요일구분'])
    .groupby(['호선', '출발역'], as_index=False)
    .mean(numeric_only=True)
)

# 시간대별 평균 승객 수 → 단순 평균값 그대로 혼잡도로 사용
df_metro_avg['혼잡도값'] = df_metro_avg.loc[:, '5시':'00시'].mean(axis=1)

# (호선, 출발역) → 혼잡도 수치 맵핑
metro_congestion_map = {
    (row['호선'], row['출발역']): row['혼잡도값']
    for _, row in df_metro_avg.iterrows()
}


# ————————————————————————————————————————————————————————
# 3. 버스 혼잡도 전처리
# ————————————————————————————————————————————————————————
# '정류소명'에서 괄호 제거
df_bus_confuse['정류소명'] = df_bus_confuse['정류소명'].str.replace(r'\(.*\)', '', regex=True).str.strip()

# 동일 (계절, 버스번호, 정류소명)에 대해 00~23시 승객 수 평균
bus_grouped = (
    df_bus_confuse
    .groupby(['season', '버스번호', '정류소명'], as_index=False)
    .mean(numeric_only=True)
)

# 혼잡도: 시간대별 평균 그대로 사용
bus_grouped['혼잡도값'] = bus_grouped.loc[:, '00시총승객수':'23시총승객수'].mean(axis=1)

# (계절, 노선, 정류소명) → 혼잡도 수치
bus_congestion_map = {
    (row['season'], row['버스번호'], row['정류소명']): row['혼잡도값']
    for _, row in bus_grouped.iterrows()
}
# ————————————————————————————————————————————————————————
# 4. 버스 속도 보정값 맵
# ————————————————————————————————————————————————————————
# 현재 계절에 맞는 속도 추출
season_speed_col = current_season  # 예: 'spring'

# 결측값 제거 및 숫자형 변환
df_bus_speed = df_bus_speed.dropna(subset=[season_speed_col])
df_bus_speed[season_speed_col] = pd.to_numeric(df_bus_speed[season_speed_col], errors='coerce')

# 기준 속도 22.7km/h 대비한 보정값 = 22.7 / 계절별 속도
df_bus_speed['보정값'] = 22.7 / df_bus_speed[season_speed_col]

# (lat, lng) → 보정값 맵
bus_speed_map = {
    (row['lat'], row['lng']): row['보정값']
    for _, row in df_bus_speed.iterrows()
}
# 전처리된 맵 3개 반환
metro_congestion_map, bus_congestion_map, bus_speed_map


({('1호선', '동대문'): 27.75791666666667,
  ('1호선', '동묘앞'): 25.94375,
  ('1호선', '서울'): 30.338750000000005,
  ('1호선', '시청'): 28.98291666666666,
  ('1호선', '신설동'): 24.668333333333337,
  ('1호선', '제기동'): 22.861666666666665,
  ('1호선', '종각'): 28.06708333333333,
  ('1호선', '종로3가'): 28.374166666666667,
  ('1호선', '종로5가'): 28.595416666666672,
  ('1호선', '청량리'): 20.160833333333336,
  ('2호선', '강남'): 40.517916666666665,
  ('2호선', '강변'): 27.734583333333337,
  ('2호선', '건대입구'): 29.740833333333335,
  ('2호선', '교대'): 42.17,
  ('2호선', '구로디지털단지'): 34.15541666666666,
  ('2호선', '구의'): 28.4475,
  ('2호선', '까치산'): 6.099166666666667,
  ('2호선', '낙성대'): 39.937916666666666,
  ('2호선', '당산'): 38.950833333333335,
  ('2호선', '대림'): 36.05541666666666,
  ('2호선', '도림천'): 19.997499999999995,
  ('2호선', '동대문역사문화공원'): 38.608333333333334,
  ('2호선', '뚝섬'): 30.750416666666673,
  ('2호선', '문래'): 38.21333333333333,
  ('2호선', '방배'): 42.6425,
  ('2호선', '봉천'): 34.95583333333333,
  ('2호선', '사당'): 41.93000000000001,
  ('2호선', '삼성'): 30.419999999

In [ ]:
def inject_congestion_to_graph(G, season, metro_map, bus_map, speed_map):
    """
    season: 'spring', 'summer', ...
    G: 통합 그래프
    metro_map: (호선, 출발역) → 혼잡도 점수
    bus_map: (season, 버스번호, 정류소명) → 혼잡도 점수
    speed_map: {season: {(lat, lng): 속도보정값}}
    """
    speed_data = speed_map.get(season, {})

    for node, data in G.nodes(data=True):
        mode = data.get('mode')
        lat, lng = data.get('lat'), data.get('lng')
        if lat is None or lng is None:
            continue

        congestion_score = 2  # 기본값

        if mode == 'metro':
            line = data.get('line')
            station = data.get('station')
            congestion_score = metro_map.get((line, station), 2)

        elif mode == 'bus':
            route = str(data.get('route'))
            stop_name = str(data.get('node_key')).split('_', 1)[-1].strip()

            bus_score = bus_map.get((season, route, stop_name), 2)
            # 속도 보정값 (가장 가까운 좌표 찾기)
            latlng_key = (round(lat, 5), round(lng, 5))
            speed_score = speed_data.get(latlng_key, 1.0)

            # 혼잡도 × 속도보정
            congestion_score = bus_score * speed_score

        G.nodes[node]['congestion_score'] = congestion_score

# 적용 예시
from datetime import datetime
from zoneinfo import ZoneInfo

def get_current_season():
    m = datetime.now(ZoneInfo("Asia/Seoul")).month
    if 3 <= m <= 5: return 'spring'
    if 6 <= m <= 8: return 'summer'
    if 9 <= m <= 11: return 'fall'
    return 'winter'

current_season = get_current_season()
inject_congestion_to_graph(integrated_graph, current_season,
                           metro_congestion_map, bus_congestion_map, bus_speed_map)


In [ ]:
def evaluate_route_congestion(path, G):
    """경로 내 혼잡도 점수 평균"""
    scores = []
    for u, v in zip(path, path[1:]):
        score = G[u][v].get('congestion_score', 2.0)  # 기본값 2.0
        scores.append(score)
    return np.mean(scores)
def find_least_congested_routes(G, origin_cand, dest_cand, k=5, max_transfers=4):
    # (1) 시간 기준 상위 N개 경로 먼저 뽑기
    time_routes = get_fastest_routes_optimized(G, origin_cand, dest_cand,
                                               k=50, max_transfers=max_transfers)
    # (2) 각 경로에 혼잡 점수 부여
    for r in time_routes:
        r['congestion_score'] = evaluate_route_congestion(r['path'], G)

    # (3) 혼잡 점수 오름차순 정렬 (작을수록 덜 혼잡함)
    sorted_routes = sorted(time_routes, key=lambda x: x['congestion_score'])
    return sorted_routes[:k]


In [ ]:
import networkx as nx

# ————————————————————————————————————————————————————————
# 5. 혼잡도 계산 함수 정의
# ————————————————————————————————————————————————————————

def get_congestion_score(u, v, G, season=current_season):
    mode = G[u][v].get('mode')

    if mode == 'metro_edge':
        line = G.nodes[u].get('line')
        station = G.nodes[u].get('station')
        return metro_congestion_map.get((line, station), 2.0)

    elif mode == 'bus_edge':
        route = G.nodes[u].get('route')
        stop_name = G.nodes[u].get('node_key', '').split("_", 1)[-1]
        season_key = season
        congestion = bus_congestion_map.get((season_key, route, stop_name), 2.0)

        # 위치 기반 보정값
        lat, lng = G.nodes[u].get('lat'), G.nodes[u].get('lng')
        correction = None
        if lat is not None and lng is not None:
            correction = bus_speed_map.get((lat, lng), 1.0)
        return congestion * correction if correction else congestion

    elif mode == 'transfer':
        return 1.0  # 낮은 혼잡도로 고정

    else:
        return 2.0  # 기본 중간값


# ————————————————————————————————————————————————————————
# 6. 경로 혼잡도 점수 계산
# ————————————————————————————————————————————————————————
def evaluate_congestion_for_route(path, G):
    scores = [get_congestion_score(u, v, G) for u, v in zip(path, path[1:])]
    return np.mean(scores)


# ————————————————————————————————————————————————————————
# 7. 혼잡도 기준 경로 탐색
# ————————————————————————————————————————————————————————
def find_least_congested_routes(G, origin_candidates, dest_candidates,
                                 k=5, max_transfers=4):
    time_routes = get_fastest_routes_optimized(G, origin_candidates, dest_candidates,
                                               k=50, max_transfers=max_transfers)
    for r in time_routes:
        r['congestion_score'] = evaluate_congestion_for_route(r['path'], G)

    return sorted(time_routes, key=lambda x: x['congestion_score'])[:k]


In [ ]:
congested_routes = find_least_congested_routes(
    integrated_graph, start_candidates, end_candidates, k=100
)

for i, r in enumerate(congested_routes, 1):
    print(f"\n[{i}위] 혼잡도점수={r['congestion_score']:.2f}, "
          f"총시간={r['time_min']:.1f}분 (보행={r['walk_min']:.1f}분, 환승={r['transfers']}회)")
    print("경로:", " → ".join(r['path']))



[1위] 혼잡도점수=17.68, 총시간=49.2분 (보행=13.5분, 환승=0회)
경로: 741_종로2가.삼일교 → 741_종로2가 → 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[2위] 혼잡도점수=18.30, 총시간=47.3분 (보행=14.1분, 환승=0회)
경로: 741_종로2가 → 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[3위] 혼잡도점수=19.52, 총시간=48.3분 (보행=16.1분, 환승=0회)
경로: 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[4위] 혼잡도점수=19.73, 총시간=56.7분 (보행=19.3분, 환승=0회)
경로: 741_남대문세무서 → 741_종로2가.삼일교 → 741_종로2가 → 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[5위] 혼잡도점수=20.78, 총시간=60.2분 (보행=17.7분, 환승=0회)
경로: 741_종로2가.삼일교 → 741_종로2가 → 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역 → 741_신분당선강남역

[6위] 혼잡도점수=20.86, 총시간=67.2분 (보행=20.8분, 환승=0회)
경로: 741_종로2가.삼일교 → 741_종로2가 → 741_종로2가사거리 

In [ ]:
# 모든 평가 함수들 실행 및 통합 정리

# 경로 추천 함수들 호출
results_combined = {
    'fastest': get_fastest_routes_optimized(integrated_graph, start_candidates, end_candidates,
                                            k=50, max_transfers=4),
    'transfer': get_transfer_convenient_routes(integrated_graph, start_candidates, end_candidates,
                                               transfer_time_map, es_map,
                                               k=50, max_transfers=4),
    'punctual': get_best_punctuality_routes(integrated_graph, start_candidates, end_candidates,
                                            df_metro_late, metro_accidents, k=50),
    'pleasant': find_most_pleasant_routes(integrated_graph, start_candidates, end_candidates,
                                          k=50),
    'congestion': find_least_congested_routes(integrated_graph, start_candidates, end_candidates,
                                              k=50)
}

# 결과 출력
for key, routes in results_combined.items():
    print(f"\n=== {key.upper()} ROUTES (Top 50) ===")
    for i, r in enumerate(routes, 1):
        score_str = ""
        if key == "fastest":
            score_str = f"총시간={r['time_min']:.1f}분"
        elif key == "transfer":
            score_str = f"환승편의성={r['score']:.2f}"
        elif key == "punctual":
            score_str = f"정시성점수={r['score']:.5f}"
        elif key == "pleasant":
            score_str = f"쾌적점수={r['pleasant_score']:.2f}"
        elif key == "congestion":
            score_str = f"혼잡도점수={r['congestion_score']:.2f}"

        print(f"[{i}] {score_str}, 총시간={r.get('time_min', 0):.1f}분 "
              f"(보행={r['walk_min']:.1f}분, 환승={r['transfers']}회)")
        print("경로:", " → ".join(r['path']))
    print()



=== FASTEST ROUTES (Top 50) ===
[1] 총시간=28.4분, 총시간=28.4분 (보행=2.9분, 환승=1회)
경로: 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남
[2] 총시간=36.8분, 총시간=36.8분 (보행=12.4분, 환승=1회)
경로: 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현
[3] 총시간=38.3분, 총시간=38.3분 (보행=14.9분, 환승=1회)
경로: 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남
[4] 총시간=39.7분, 총시간=39.7분 (보행=5.1분, 환승=2회)
경로: 5호선_종로3가 → 5호선_을지로4가 → 5호선_동대문역사문화공원 → 5호선_청구 → 5호선_신금호 → 5호선_행당 → 5호선_왕십리 → 수인분당선_왕십리 → 수인분당선_서울숲 → 수인분당선_압구정로데오 → 수인분당선_강남구청 → 수인분당선_선정릉 → 수인분당선_선릉 → 2호선_선릉 → 2호선_역삼 → 2호선_강남
[5] 총시간=41.8분, 총시간=41.8분 (보행=11.6분, 환승=3회)
경로: 5호선_을지로4가 → 5호선_동대문역사문화공원 → 5호선_청구 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남
[6] 총시간=41.9분, 총시간=41.9분 (보행=12.6분, 환승=2회)
경로: 2호선_

# 종합

정시성: 약 29.7%
환승편의: 약 29.1%
쾌적성: 약 16.2%
혼잡도: 약 14.4%
소요시간: 약 10.7%

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 1. 평가 기준별 결과 수집
top_routes_by_time = get_fastest_routes_optimized(integrated_graph, start_candidates, end_candidates, k=50)

# 각각의 점수 계산
for r in top_routes_by_time:
    r['punctuality'] = calc_route_punctuality(r['path'], integrated_graph, get_current_season(), df_metro_late, metro_accidents)
    r['transfer_score'] = r['transfers']
    r['pleasant_score'] = evaluate_route_pleasant(r['path'], integrated_graph)
    r['congestion_score'] = evaluate_route_congestion(r['path'], integrated_graph)


In [ ]:
# DataFrame으로 변환
df = pd.DataFrame(top_routes_by_time)

# 2. Min-Max 정규화 수행
scaler = MinMaxScaler()

# 낮을수록 좋은 것들 → 정규화 후 (1 - 값) 처리
cols_inverted = ['time_min', 'transfers', 'walk_min', 'punctuality', 'congestion_score']
for col in cols_inverted:
    df[f'{col}_scaled'] = 1 - scaler.fit_transform(df[[col]])

# 높을수록 좋은 것 → 그대로 정규화
df['pleasant_score_scaled'] = scaler.fit_transform(df[['pleasant_score']])

# 3. 가중치 부여
weights = {
    'punctuality_scaled': 0.297,
    'transfers_scaled': 0.291,
    'pleasant_score_scaled': 0.162,
    'congestion_score_scaled': 0.144,
    'time_min_scaled': 0.107
}

# 최종 스코어 계산
df['final_score'] = (
    df['punctuality_scaled'] * weights['punctuality_scaled'] +
    df['transfers_scaled'] * weights['transfers_scaled'] +
    df['pleasant_score_scaled'] * weights['pleasant_score_scaled'] +
    df['congestion_score_scaled'] * weights['congestion_score_scaled'] +
    df['time_min_scaled'] * weights['time_min_scaled']
)

# 점수 내림차순 정렬
df_sorted = df.sort_values(by='final_score', ascending=False).reset_index(drop=True)


In [ ]:
# 6. 출력
df_sorted = df.sort_values(by='final_score', ascending=False).reset_index(drop=True)
for i, row in df_sorted.iterrows():
    rank = i + 1
    print(f"\n[{rank}위] 종합만족도점수={row['final_score']*100:.5f}, 총시간={row['time_min']:.1f}분 "
          f"(보행={row['walk_min']:.1f}분, 환승={row['transfers']}회)")
    print("경로:", " → ".join(row['path']))
    if rank == 50:
        break


[1위] 종합만족도점수=95.90300, 총시간=44.0분 (보행=9.5분, 환승=0회)
경로: 140_종로4가.종묘 → 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[2위] 종합만족도점수=95.26986, 총시간=46.6분 (보행=13.6분, 환승=0회)
경로: 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[3위] 종합만족도점수=92.69776, 총시간=48.3분 (보행=16.1분, 환승=0회)
경로: 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[4위] 종합만족도점수=92.68223, 총시간=47.3분 (보행=14.1분, 환승=0회)
경로: 741_종로2가 → 741_종로2가사거리 → 741_국가인권위.안중근활동터 → 741_순천향대학병원 → 741_한남오거리 → 741_한남대교전망카페 → 741_신사역 → 741_논현역 → 741_지하철2호선강남역

[5위] 종합만족도점수=92.30183, 총시간=57.9분 (보행=27.6분, 환승=0회)
경로: 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_신사역 → 140_논현역 → 140_지하철2호선강남역

[6위] 종합만족도점수=92.23483, 총시간=57.8분 (보행=20.3분, 환승=0회)
경로: 140_종로5가.광장시장 → 140_종로4가.종묘 → 140_종로3가.탑골공원 → 140_국가인권위.안중근활동터 → 140_순천향대학병원 → 140_한남오거리 → 140_한남대교전망카페 → 140_